<a href="https://colab.research.google.com/github/fabriziobasso/Colab_backup/blob/main/rain_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **S4E10 - Rain Forecast**

The table below shows the column names, their value formats, and their description.

Index| Features      |Format             |Description
-----|---------------|-------------------|-----------------------
1    |Date Time      |01.01.2009 00:10:00|Date-time reference
2    |p (mbar)       |996.52             |The pascal SI derived unit of pressure used to quantify internal pressure. Meteorological reports typically state atmospheric pressure in millibars.
3    |T (degC)       |-8.02              |Temperature in Celsius
4    |Tpot (K)       |265.4              |Temperature in Kelvin
5    |Tdew (degC)    |-8.9               |Temperature in Celsius relative to humidity. Dew Point is a measure of the absolute amount of water in the air, the DP is the temperature at which the air cannot hold all the moisture in it and water condenses.
6    |rh (%)         |93.3               |Relative Humidity is a measure of how saturated the air is with water vapor, the %RH determines the amount of water contained within collection objects.
7    |VPmax (mbar)   |3.33               |Saturation vapor pressure
8    |VPact (mbar)   |3.11               |Vapor pressure
9    |VPdef (mbar)   |0.22               |Vapor pressure deficit
10   |sh (g/kg)      |1.94               |Specific humidity
11   |H2OC (mmol/mol)|3.12               |Water vapor concentration
12   |rho (g/m ** 3) |1307.75            |Airtight
13   |wv (m/s)       |1.03               |Wind speed
14   |max. wv (m/s)  |1.75               |Maximum wind speed
15   |wd (deg)       |152.3              |Wind direction in degrees

## Intro: Problem Statement

<div class="container">
        <p class="mb-2">
            Analyze weather data over a 5-day period to identify patterns and correlations between various meteorological factors such as pressure, temperature, humidity, and rainfall. This can help in understanding how these variables influence daily weather conditions.
        </p>
        <h1 class="mb-2">Dataset Description</h1>
        <p class="mb-2">The dataset captures weather data over five days with the following features:</p>
        <ul class="mb-2">
            <li class="mb-2"><strong>day:</strong> Sequential day number.</li>
            <li class="mb-2"><strong>pressure (hPa):</strong> Atmospheric pressure measured in hectopascals.</li>
            <li class="mb-2"><strong>maxtemp (°C):</strong> Maximum temperature recorded on the day.</li>
            <li class="mb-2"><strong>temperature (°C):</strong> Average temperature of the day.</li>
            <li class="mb-2"><strong>mintemp (°C):</strong> Minimum temperature recorded on the day.</li>
            <li class="mb-2"><strong>dewpoint (°C):</strong> Temperature at which air becomes saturated with moisture.</li>
            <li class="mb-2"><strong>humidity (%):</strong> Relative humidity percentage.</li>
            <li class="mb-2"><strong>cloud (%):</strong> Cloud cover percentage.</li>
            <li class="mb-2"><strong>rainfall:</strong> Indicates if rainfall occurred ("yes"/"no").</li>
            <li class="mb-2"><strong>sunshine (hours):</strong> Total sunshine hours.</li>
            <li class="mb-2"><strong>winddirection (°):</strong> Direction of the wind in degrees.</li>
            <li class="mb-2"><strong>windspeed (km/h):</strong> Speed of the wind.</li>
        </ul>
        <p class="mb-2">
            This dataset is ideal for basic weather pattern analysis and visualization.
        </p>
    </div>

## 1.0 Libraries

In [ ]:
%%capture
# #!pip install -qq pytorch_tabnet
!pip install optuna
# !pip install catboost
# #!pip install --upgrade numpy
# #!pip install optuna-integration-pytorch-tabnet

# #from pytorch_tabnet.tab_model import TabNetRegressor

!pip install --upgrade category-encoders
!pip install optuna-integration
!pip install colorama
# #!pip install pyfiglet
!pip install keras-tuner --upgrade
# !pip install keras-nlp
# !pip install BorutaShap
# !pip install --upgrade scikit-learn
# !pip install scikit-lego
!pip install skops

# #from pytorch_tabnet.tab_model import TabNetRegressor

In [ ]:
# Setup notebook
from pathlib import Path
import ipywidgets as widgets
import pandas as pd
import numpy as np
from pickle import load, dump
import json
import joblib
import skops.io as sio
#import calplot as cal

# Graphic Libraries:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
# Set Style
sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5});
sns.despine(left=True, bottom=True, top=False, right=False);
mpl.rcParams['figure.dpi'] = 120;
mpl.rc('axes', labelsize=12);
plt.rc('xtick',labelsize=10);
plt.rc('ytick',labelsize=10);

mpl.rcParams['axes.spines.top'] = False;
mpl.rcParams['axes.spines.right'] = False;
mpl.rcParams['axes.spines.left'] = True;

# Palette Setup
colors = ['#FB5B68','#FFEB48','#2676A1','#FFBDB0',]
colormap_0 = mpl.colors.LinearSegmentedColormap.from_list("",colors)
palette_1 = sns.color_palette("coolwarm", as_cmap=True)
palette_2 = sns.color_palette("YlOrBr", as_cmap=True)
palette_3 = sns.light_palette("red", as_cmap=True)
palette_4 = sns.color_palette("viridis", as_cmap=True)
palette_5 = sns.color_palette("rocket", as_cmap=True)
palette_6 = sns.color_palette("GnBu", as_cmap=True)
palette_7 = sns.color_palette("tab20c", as_cmap=False)
palette_8 = sns.color_palette("Set2", as_cmap=False)

palette_custom = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec','#f2f2f2']
palette_9 = sns.color_palette(palette_custom, as_cmap=False)

# tool for Excel:
from openpyxl import load_workbook, Workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.worksheet.cell_range import CellRange

from openpyxl.formatting import Rule
from openpyxl.styles import Font, PatternFill, Border
from openpyxl.styles.differential import DifferentialStyle

#from catboost import CatBoostRegressor, Pool, CatBoostClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from xgboost.callback import EarlyStopping

import lightgbm as lgb
from lightgbm import (LGBMRegressor,
                      LGBMClassifier,
                      early_stopping,
                      record_evaluation,
                      log_evaluation)

# Time Management
from tqdm import tqdm
from datetime import date
from datetime import datetime
from pandas.tseries.offsets import BMonthEnd, QuarterEnd
from pandas.tseries.offsets import BDay # BDay is business day, not birthday...
import datetime as dt
import click
import glob
import os
import gc
import re
import string

from ipywidgets import AppLayout
from ipywidgets import Dropdown, Layout, HTML, AppLayout, VBox, Label, HBox, BoundedFloatText, interact, Output

#from my_func import *

import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

from keras.layers import Input, LSTM, Dense, Lambda, RepeatVector, Reshape
from keras.models import Model
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

from keras.utils import FeatureSpace, plot_model

# Import libraries for Hypertuning
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch, GridSearch, BayesianOptimization

#from my_func import *

# preprocessing modules
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedKFold, cross_val_score, cross_validate, GroupKFold, GridSearchCV, RepeatedStratifiedKFold, cross_val_predict
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

from sklearn.preprocessing import (LabelEncoder,
                                   StandardScaler,
                                   MinMaxScaler,
                                   OrdinalEncoder,
                                   RobustScaler,
                                   PowerTransformer,
                                   OneHotEncoder,
                                   QuantileTransformer,
                                   PolynomialFeatures)

# metrics
import sklearn
#import skops.io as sio
from sklearn.metrics import (mean_squared_error,
                             root_mean_squared_error,
                             root_mean_squared_log_error,
                             r2_score,
                             mean_absolute_error,
                             mean_absolute_percentage_error,
                             classification_report,
                             confusion_matrix,
                             ConfusionMatrixDisplay,
                             multilabel_confusion_matrix,
                             accuracy_score,
                             roc_auc_score,
                             auc,
                             roc_curve,
                             log_loss,
                             make_scorer)
# modeling algos
from sklearn.linear_model import (LogisticRegression,
                                  Lasso,
                                  ridge_regression,
                                  LinearRegression,
                                  Ridge,
                                  RidgeCV,
                                  ElasticNet,
                                  BayesianRidge,
                                  HuberRegressor,
                                  TweedieRegressor,
                                  QuantileRegressor,
                                  ARDRegression,
                                  TheilSenRegressor,
                                  PoissonRegressor,
                                  GammaRegressor)

from sklearn.ensemble import (AdaBoostRegressor,
                              AdaBoostClassifier,
                              RandomForestRegressor,
                              RandomForestClassifier,
                              VotingRegressor,
                              GradientBoostingRegressor,
                              GradientBoostingClassifier,
                              StackingRegressor,
                              StackingClassifier,
                              HistGradientBoostingClassifier,
                              HistGradientBoostingRegressor,
                              ExtraTreesClassifier)

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

import itertools
import warnings
from openpyxl import load_workbook

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor

import statsmodels.api as sm
from pylab import rcParams
import scipy.stats as ss

from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.wrapper import PolynomialWrapper
from category_encoders.count import CountEncoder
from category_encoders import TargetEncoder

warnings.filterwarnings('ignore')
#import pyfiglet
#plt.style.use('fivethirtyeight')

In [ ]:
sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.5,
         'grid.alpha'           :0.5,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : 'black',
         'axes.titlesize'       : 12,
#         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        });

sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5})
# Set Style
mpl.rcParams['figure.dpi'] = 120;

# import font colors
from colorama import Fore, Style, init

# Making sklearn pipeline outputs as dataframe:-
pd.set_option('display.max_columns', 100);
pd.set_option('display.max_rows', 50);

sns.despine(left=True, bottom=True, top=False, right=False)

mpl.rcParams['axes.spines.left'] = True
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = True

init(autoreset=True)

In [ ]:
from tqdm import tqdm
from itertools import product

import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor

from sklearn.model_selection import GroupKFold
from sklearn.impute import SimpleImputer
import torch

import warnings
warnings.filterwarnings("ignore")

# Connect to Colab:#
from google.colab import drive
drive.mount('/content/drive')

## 2.0 Loading and Preprocessing Data


In [ ]:
df_subm = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/sample_submission.csv", index_col=0)

validation = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/X_enc_y_old_all_ext.csv")
train = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/X_enc_y_new_all_ext.csv")
test = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/test_enc_all_ext.csv")

In [ ]:
#df_train_old.isna().sum()

In [ ]:
train.shape, test.shape, validation.shape

In [ ]:
df_all_new = pd.concat([train, test], axis=0)
print(df_all_new.shape)
df_all_new.head(3)

In [ ]:
df_all_new.tail(3)

In [ ]:
step = 1

past = 7
future = 0
learning_rate = 0.01
batch_size = 64
epochs = 21

test_final = df_all_new.iloc[2190-past+1:,:]

In [ ]:
# Train data
x_train = train.iloc[:,:-1].values
y_train = train.iloc[past-1:,-1].values.reshape(-1,1)
print(x_train.shape, y_train.shape)
# Valid Data
x_valid = validation.iloc[:,:-1].values
y_valid = validation.iloc[past-1:,-1].values.reshape(-1,1)
print(x_valid.shape, y_valid.shape)
# Test Data
x_test = test_final.iloc[:,:-1].values
y_test = test_final.iloc[past-1:,-1].values.reshape(-1,1)
print(x_test.shape, y_test.shape)

## 3.0 Dataset Management Functions

## Version 0

####  **3.1 Train Dataset**

In [ ]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_train,
                                                                    y_train,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10000,
                                                                    shuffle=True
                                                                 )


for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

In [ ]:
# Find indices of 0s
indices_of_zeros = np.where(targets.numpy() == 0)[0]

# Find indices of 1s
indices_of_ones = np.where(targets.numpy() == 1)[0]

In [ ]:
indices_of_zeros.shape,indices_of_ones.shape

* **TRAIN DATASET - INFINITE SAMPLING**

* **GOAL: Oversample the minority class**
A related approach would be to resample the dataset by oversampling the minority class.

In [ ]:
pos_features = inputs.numpy()[indices_of_ones]
neg_features = inputs.numpy()[indices_of_zeros]

pos_labels = targets.numpy()[indices_of_ones]
neg_labels = targets.numpy()[indices_of_zeros]

pos_features.shape,pos_labels.shape,neg_features.shape,neg_labels.shape

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

In [ ]:
for num, (features, label) in enumerate(pos_ds.take(10)):
  #print("Features:\n", features.numpy())
  #print()
  print(f"Label sample {num}: ", label.numpy())

In [ ]:
resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(64).prefetch(2)

In [ ]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

#### **3.2 Validation Dataset**

In [ ]:
dataset_validation = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_valid,
                                                                    y_valid,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=64,
                                                                    shuffle=False
                                                                 )


for batch in dataset_validation.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

#### **3.3 Test Dataset**

In [ ]:
dataset_test = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_test,
                                                                    y_test,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=64,
                                                                    shuffle=False
                                                                 )


for batch in dataset_test.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

## Version 1

In [ ]:
dataset_validation = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_valid,
                                                                    y_valid,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10_000,
                                                                    shuffle=False
                                                                 )


for batch in dataset_validation.take(1):
    inputs_val, targets_val = batch

print("Input shape:", inputs_val.numpy().shape)
print("Target shape:", targets_val.numpy().shape)

In [ ]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_train,
                                                                    y_train,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10000,
                                                                    shuffle=True
                                                                 )


for batch in dataset_train.take(1):
    inputs_tr, targets_tr = batch

print("Input shape:", inputs_tr.numpy().shape)
print("Target shape:", targets_tr.numpy().shape)

In [ ]:
# Merge the arrays along the first axis
all_train = np.concatenate((inputs_val, inputs_tr), axis=0)
all_target = np.concatenate((targets_val, targets_tr), axis=0)
all_train.shape, all_target.shape

In [ ]:
all_train.shape[0]-365, all_train.shape

In [ ]:
# Train data
x_train_v1 = all_train[:2178,:,:]
y_train_v1 = all_target[:2178,:]
print(x_train_v1.shape, y_train_v1.shape)
# Valid Data
x_valid_v1 = all_train[2178:,:,:]
y_valid_v1 = all_target[2178:,:]
print(x_valid_v1.shape, y_valid_v1.shape)
# Test Data
x_test_v1 = test_final.iloc[:,:-1].values
y_test_v1 = test_final.iloc[past-1:,-1].values.reshape(-1,1)
print(x_test_v1.shape, y_test_v1.shape)

####  **3.1 Train Dataset**

In [ ]:
dataset_train_v1 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_train_v1,
                                                                    y_train_v1,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10000,
                                                                    shuffle=True
                                                                 )


for batch in dataset_train_v1.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

In [ ]:
# Convert numpy arrays to TensorFlow datasets
feature_dataset = tf.data.Dataset.from_tensor_slices(x_train_v1)
target_dataset = tf.data.Dataset.from_tensor_slices(y_train_v1)

# Combine features and target into a single dataset
dataset_tr = tf.data.Dataset.zip((feature_dataset, target_dataset))

# Shuffle and batch the dataset
dataset_train_v1 = dataset_tr.shuffle(buffer_size=len(x_train_v1)).batch(10000)


for batch in dataset_train_v1.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

In [ ]:
# Find indices of 0s
indices_of_zeros = np.where(targets.numpy() == 0)[0]

# Find indices of 1s
indices_of_ones = np.where(targets.numpy() == 1)[0]

In [ ]:
indices_of_zeros.shape,indices_of_ones.shape

* **TRAIN DATASET - INFINITE SAMPLING**

* **GOAL: Oversample the minority class**
A related approach would be to resample the dataset by oversampling the minority class.

In [ ]:
pos_features_v1 = inputs.numpy()[indices_of_ones]
neg_features_v1 = inputs.numpy()[indices_of_zeros]

pos_labels_v1 = targets.numpy()[indices_of_ones]
neg_labels_v1 = targets.numpy()[indices_of_zeros]

pos_features_v1.shape,pos_labels_v1.shape,neg_features_v1.shape,neg_labels_v1.shape

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds_v1 = make_ds(pos_features_v1, pos_labels_v1)
neg_ds_v1 = make_ds(neg_features_v1, neg_labels_v1)

In [ ]:
for num, (features, label) in enumerate(pos_ds_v1.take(10)):
  #print("Features:\n", features.numpy())
  #print()
  print(f"Label sample {num}: ", label.numpy(), features.shape)

In [ ]:
resampled_ds_v1 = tf.data.Dataset.sample_from_datasets([pos_ds_v1, neg_ds_v1], weights=[0.5, 0.5])
resampled_ds_v1 = resampled_ds_v1.batch(64).prefetch(2)

In [ ]:
for features, label in resampled_ds_v1.take(1):
  print(label.shape,label.numpy().mean())

In [ ]:
x_valid_v1.shape, y_valid_v1.shape, x_valid.shape, y_valid.shape

#### **3.2 Validation Dataset**

In [ ]:
feature_dataset = tf.data.Dataset.from_tensor_slices(x_valid_v1)
target_dataset = tf.data.Dataset.from_tensor_slices(y_valid_v1)

# Combine features and target into a single dataset
dataset_validation_v1 = tf.data.Dataset.zip((feature_dataset, target_dataset))

# Shuffle and batch the dataset
dataset_validation_v1 = dataset_validation_v1.batch(64).prefetch(2)

for batch in dataset_validation_v1.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

#### **3.3 Test Dataset**

In [ ]:
dataset_test_v1 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_test,
                                                                    y_test,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=64,
                                                                    shuffle=False
                                                                 )


for batch in dataset_test_v1.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

## Version 2

In [ ]:
dataset_validation_v2 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_valid,
                                                                    y_valid,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10_000,
                                                                    shuffle=False
                                                                 )


for batch in dataset_validation_v2.take(1):
    inputs_val_v2, targets_val_v2 = batch

print("Input shape:", inputs_val_v2.numpy().shape)
print("Target shape:", targets_val_v2.numpy().shape)

In [ ]:
dataset_train_v2 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_train,
                                                                    y_train,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10000,
                                                                    shuffle=True
                                                                 )


for batch in dataset_train_v2.take(1):
    inputs_tr_v2, targets_tr_v2 = batch

print("Input shape:", inputs_tr_v2.numpy().shape)
print("Target shape:", targets_tr_v2.numpy().shape)

In [ ]:
# Merge the arrays along the first axis
all_train_v2 = np.concatenate((inputs_val_v2, inputs_tr_v2), axis=0)
all_target_v2 = np.concatenate((targets_val_v2, targets_tr_v2), axis=0)
all_train_v2.shape, all_target_v2.shape

In [ ]:
all_train.shape[0]-365, all_train.shape

In [ ]:
x_train_v2, x_valid_v2, y_train_v2, y_valid_v2 = train_test_split(all_train, all_target, test_size= 0.2, random_state=42, shuffle=True, stratify=all_target)
y_train_v2 = y_train_v2.reshape(-1,1)
y_valid_v2 = y_valid_v2.reshape(-1,1)

print(x_train_v2.shape, y_train_v2.shape)
print(x_valid_v2.shape, y_valid_v2.shape)
# Test Data
x_test_v2 = test_final.iloc[:,:-1].values
y_test_v2 = test_final.iloc[past-1:,-1].values.reshape(-1,1)
print(x_test_v2.shape, y_test_v2.shape)

In [ ]:
all_train.shape

####  **3.1 Train Dataset**

In [ ]:
# Convert numpy arrays to TensorFlow datasets
feature_dataset_v2 = tf.data.Dataset.from_tensor_slices(x_train_v2)
target_dataset_v2 = tf.data.Dataset.from_tensor_slices(y_train_v2)

# Combine features and target into a single dataset
dataset_tr_v2 = tf.data.Dataset.zip((feature_dataset_v2, target_dataset_v2))

# Shuffle and batch the dataset
dataset_train_v2 = dataset_tr_v2.shuffle(buffer_size=len(x_train_v2)).batch(10000)


for batch in dataset_train_v2.take(1):
    inputs_v2, targets_v2 = batch

print("Input shape:", inputs_v2.numpy().shape)
print("Target shape:", targets_v2.numpy().shape)

In [ ]:
# Find indices of 0s
indices_of_zeros_v2 = np.where(targets_v2.numpy() == 0)[0]

# Find indices of 1s
indices_of_ones_v2 = np.where(targets_v2.numpy() == 1)[0]

In [ ]:
indices_of_zeros_v2.shape,indices_of_ones_v2.shape

* **TRAIN DATASET - INFINITE SAMPLING**

* **GOAL: Oversample the minority class**
A related approach would be to resample the dataset by oversampling the minority class.

In [ ]:
pos_features_v2 = inputs_v2.numpy()[indices_of_ones_v2]
neg_features_v2 = inputs_v2.numpy()[indices_of_zeros_v2]

pos_labels_v2 = targets_v2.numpy()[indices_of_ones_v2]
neg_labels_v2 = targets_v2.numpy()[indices_of_zeros_v2]

pos_features_v2.shape,pos_labels_v2.shape,neg_features_v2.shape,neg_labels_v2.shape

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features_v2, labels_v2):
  ds_v2 = tf.data.Dataset.from_tensor_slices((features_v2, labels_v2))#.cache()
  ds_v2 = ds_v2.shuffle(BUFFER_SIZE).repeat()
  return ds_v2

pos_ds_v2 = make_ds(pos_features_v2, pos_labels_v2)
neg_ds_v2 = make_ds(neg_features_v2, neg_labels_v2)

In [ ]:
for num, (features, label) in enumerate(pos_ds_v2.take(10)):
  #print("Features:\n", features.numpy())
  #print()
  print(f"Label sample {num}: ", label.numpy(), features.shape)

In [ ]:
resampled_ds_v2 = tf.data.Dataset.sample_from_datasets([pos_ds_v2, neg_ds_v2], weights=[0.5, 0.5])
resampled_ds_v2 = resampled_ds_v2.batch(64).prefetch(2)

In [ ]:
for features, label in resampled_ds_v2.take(1):
  print(label.shape,label.numpy().mean())

In [ ]:
x_valid_v2.shape, y_valid_v2.shape, x_valid_v1.shape, y_valid_v1.shape

#### **3.2 Validation Dataset**

In [ ]:
feature_dataset_v2 = tf.data.Dataset.from_tensor_slices(x_valid_v2)
target_dataset_v2 = tf.data.Dataset.from_tensor_slices(y_valid_v2)

# Combine features and target into a single dataset
dataset_validation_v2 = tf.data.Dataset.zip((feature_dataset_v2, target_dataset_v2))

# Shuffle and batch the dataset
dataset_validation_v2 = dataset_validation_v2.batch(64).prefetch(2)

for batch in dataset_validation_v2.take(1):
    inputs_v2, targets_v2 = batch

print("Input shape:", inputs_v2.numpy().shape)
print("Target shape:", targets_v2.numpy().shape)

#### **3.3 Test Dataset**

In [ ]:
dataset_test_v2 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_test,
                                                                    y_test,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=64,
                                                                    shuffle=False
                                                                 )


for batch in dataset_test_v2.take(1000):
    inputs_v2, targets_v2 = batch

print("Input shape:", inputs_v2.numpy().shape)
print("Target shape:", targets_v2.numpy().shape)

## 4.0 NN Models:

In [ ]:
def plot_metrics(history):
  metrics = ['loss', 'auc']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0,1])
    else:
      plt.ylim([0,1])

    plt.legend()

def plot_cm(labels, predictions, threshold=0.5):
  cm = confusion_matrix(labels, predictions > threshold)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(threshold))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

In [ ]:
step = 1

past = 7
future = 0
learning_rate = 0.01
batch_size = 64
epochs = 101

In [ ]:
METRICS = [
          keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
          ]

### 4.1 LSTM v0

In [ ]:
def make_model_lstm(metrics=METRICS, units_rep=64, repeat=3,  units_last=32, output_bias=0, kr_lstm=0.0001, sdo=0.3, rdo=0.3, stddev=0.05):

  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  x = keras.layers.GaussianNoise(stddev=stddev, name="noise_layer")(data)


  for i in range(repeat):
    x = keras.layers.LSTM(units=units_rep,
                          return_sequences=True,
                          name=f"lstm_{i}",
                          kernel_regularizer = keras.regularizers.l2(kr_lstm),
                          dropout=sdo,
                          recurrent_dropout=rdo)(x)

  lstm_out = keras.layers.LSTM(units_last, name="lstm_final")(x)

  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(lstm_out)

  model = keras.Model(inputs=data, outputs=outputs, name="LSTM_v0")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)



  return model

model= make_model_lstm()

model.summary()

In [ ]:
train_ds_all = [resampled_ds,resampled_ds_v1,resampled_ds_v2]
validation_ds_all = [dataset_validation,dataset_validation_v1,dataset_validation_v2]
test_ds_all = [dataset_test,dataset_test_v1,dataset_test_v2]

dss = zip(train_ds_all,validation_ds_all)

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_lstm(trial, train_data, validation_data, model=make_model_lstm, use_gpu=False, rs=42, fit_scaling=False):

    model_class = model

    params = {
              'units_rep': trial.suggest_categorical('units_rep', [32,64,128,256]),
              'repeat': trial.suggest_categorical('repeat', [1,2,3,4]),
              'units_last': trial.suggest_categorical('units_last', [32,64,128,256]),
              'stddev': trial.suggest_float('stddev', 0.01, 0.1, step=0.01),
              'kr_lstm': trial.suggest_float('kr_lstm', 0.001, 0.1, log=True),
              'sdo' : trial.suggest_float('sdo', 0.20, 0.50, step=0.01),
              'rdo' : trial.suggest_float('rdo', 0.20, 0.50, step=0.01)
              }

    auc_scores = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    for count, (train_data, validation_data) in enumerate(dss):
      print(f"Fold {count+1}")

      # Fit the model
      model.fit(train_data,
                validation_data=validation_data,
                epochs=201,
                steps_per_epoch=45,
                callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                          keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                        start_from_epoch=3, mode="max")]
                )

      # Make predictions on the validation set
      y_pred = model.predict(validation_data)

      # Calculate the RMSE for the current fold
      auc_score = roc_auc_score(y_valid, y_pred)
      auc_scores.append(auc_score)

    return np.mean(auc_scores)

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=make_model_lstm, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_lstm, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - auc: 0.8906 - cross entropy: 0.4174 - loss: 0.4391 - prc: 0.8752 - val_auc: 0.8791 - val_cross entropy: 0.4404 - val_loss: 0.4584 - val_prc: 0.9343 - learning_rate: 5.0000e-04
Epoch 7/201
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - auc: 0.8844 - cross entropy: 0.4280 - loss: 0.4455 - prc: 0.8824 - val_auc: 0.8792 - val_cross entropy: 0.4652 - val_loss: 0.4822 - val_prc: 0.9334 - learning_rate: 5.0000e-04
Epoch 8/201
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - auc: 0.8868 - cross entropy: 0.4257 - loss: 0.4422 - prc: 0.8749 - val_auc: 0.8797 - val_cross entropy: 0.4509 - val_loss: 0.4666 - val_prc: 0.9330 - learning_rate: 2.5000e-04
Epoch 9/201
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - auc: 0.8910 - cross entropy: 0.4162 - loss: 0.4318 - prc: 0.8756 - val_auc: 0.8783 - val_cross entropy: 0.4652 - val_loss: 0.4804 - val_prc: 0.9317 - learning_rate: 2.5000e-04
Epoch 10/201
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - auc: 0.8936 - cross entropy: 0.4140 -

#### Fit the model

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history = model.fit(
                              resampled_ds,
                              epochs=epochs,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history)

#### Evaluate Model:

In [ ]:
train_predictions_resampled = model.predict(dataset_train)
valid_predictions_resampled = model.predict(dataset_validation)
test_predictions_resampled = model.predict(dataset_test)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
df_subm["rainfall"] = test_predictions_resampled
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_v2_all_data.csv")

### 4.2 LSTM-Tab v0

In [ ]:
def make_model(metrics=METRICS, units=[32,32],units_tab=[64,32], output_bias=None, gn=0.025, activation="relu", do=0.3):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  data_tabular = data[:, 6, :]
  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  lstm_out = keras.layers.LSTM(units[0], return_sequences=True, name="lstm_0",)(data_noised)
  lstm_out = keras.layers.LSTM(units[1], name="lstm_1")(lstm_out)

  # Tabular Section
  tabx = keras.layers.Dense(units_tab[0], name="dense_0")(data_tabular)
  tabx = keras.layers.BatchNormalization(name="batch_0")(tabx)
  tabx = keras.layers.Activation(activation, name="act_0")(tabx)
  tabx = keras.layers.Dropout(do, name="do_0")(tabx)
  tabx = keras.layers.Dense(units_tab[1], name="dense_1")(tabx)
  tabx = keras.layers.BatchNormalization(name="batch_1")(tabx)
  tabx = keras.layers.Activation(activation, name="act_1")(tabx)
  tabx = keras.layers.Dropout(do, name="do_1")(tabx)

  # Concatenate
  x = keras.layers.Concatenate(name="concat")([lstm_out, tabx,data_tabular])
  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="LSTM_tab_v1")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model(units=[64,32],output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              'activation': trial.suggest_categorical('activation', ["relu","selu","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=make_model, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

 * Trial 21 finished with value: 0.8943167305236271
    * parameters: {'units_0': 64, 'units_1': 64, 'units_tab_0': 64, 'units_tab_1': 32, 'activation': 'silu', 'gn': 0.09, 'do': 0.30781160547467370}.

 * Trial 41 finished with value: 0.8956648219100326
    * parameters: {'units_0': 128, 'units_1': 64, 'units_tab_0': 128, 'units_tab_1': 32, 'activation': 'selu', 'gn': 0.09, 'do': 0.25}.

* Best is trial 25 with value: 0.8865119909181212.
    * {'units_0': 64,
 'units_1': 32,
 'units_tab_0': 256,
 'units_tab_1': 128,
 'activation': 'selu',
 'gn': 0.08,
 'do': 0.32}


##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              'activation': trial.suggest_categorical('activation', ["relu","selu","gelu","silu","mish"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model, n_trials=111, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

#### Fit the model

##### Model 00

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
model= make_model(units=[64,64], units_tab=[128,32], activation="gelu", gn=0.09, do=0.39, output_bias=0) # Model v3 (best 0.86323)

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history = model.fit(
                              resampled_ds,
                              epochs=epochs,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, monitor="val_auc", min_lr=0.000001),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history)

##### Model 01

In [ ]:
model_cv1= make_model(units=[128,64], units_tab=[128,128], activation="mish", gn=0.09, do=0.24, output_bias=0)

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3,  factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
plot_metrics(resampled_history_v1)

#### Evaluate Model:

In [ ]:
train_predictions_resampled_v0 = model.predict(dataset_train)
valid_predictions_resampled_v0 = model.predict(dataset_validation)
test_predictions_resampled_v0 = model.predict(dataset_test)

In [ ]:
train_predictions_resampled_v1 = model_cv1.predict(dataset_train)
valid_predictions_resampled_v1 = model_cv1.predict(dataset_validation)
test_predictions_resampled_v1 = model_cv1.predict(dataset_test)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
plt.scatter(test_predictions_resampled_v0,test_predictions_resampled_v1)

In [ ]:
df_subm["rainfall"] = test_predictions_resampled_v0
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_v4_all_data_ext_v0.csv")
df_subm["rainfall"] = test_predictions_resampled_v1
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_v4_all_data_ext_v1.csv")
df_subm["rainfall"] = (test_predictions_resampled_v0+test_predictions_resampled_v1)/2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_v4_all_data_ext_average.csv")
df_subm

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_v0.shape, test_predictions_resampled_v1.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1)/2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_enhanced_v0_all_data_ext.csv")
df_subm

### 4.3 LSTM-Tab with Transformer v0

In [ ]:
inputs.shape

In [ ]:
def make_model(metrics=METRICS, units=[32,32],units_tab=[64,32], output_bias=None, gn=0.025, activation="relu", do=0.3, lr=5e-4):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  data_tabular = data[:, 6, :]
  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  whole_seq_output, final_memory_state_0, final_carry_state_0 = keras.layers.LSTM(units[0], return_sequences=True, name="lstm_0",return_state=True)(data_noised)
  whole_seq_output, final_memory_state_1, final_carry_state_1 = keras.layers.LSTM(units[1], name="lstm_1",return_state=True)(whole_seq_output)

  print(final_memory_state_0.shape, final_carry_state_0.shape)
  print(final_memory_state_1.shape, final_carry_state_1.shape)

  # Tabular Section
  data_tabular = keras.layers.Concatenate(name="concat_states")([data_tabular,final_memory_state_0,final_carry_state_0,final_memory_state_1,final_carry_state_1])

  tabx = keras.layers.Dense(units_tab[0], name="dense_0")(data_tabular)
  tabx = keras.layers.BatchNormalization(name="batch_0")(tabx)
  tabx = keras.layers.Activation(activation, name="act_0")(tabx)
  tabx = keras.layers.Dropout(do, name="do_0")(tabx)
  tabx = keras.layers.Dense(units_tab[1], name="dense_1")(tabx)
  tabx = keras.layers.BatchNormalization(name="batch_1")(tabx)
  tabx = keras.layers.Activation(activation, name="act_1")(tabx)
  tabx = keras.layers.Dropout(do, name="do_1")(tabx)

  # Concatenate
  x = keras.layers.Concatenate(name="concat")([whole_seq_output, tabx,data_tabular])
  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="LSTM_tab_v2")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=lr),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model(units=[64,32],output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

In [ ]:
#plot_model(model, show_shapes=True)

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              'activation': trial.suggest_categorical('activation', ["relu","mish","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

 * Trial 85 finished with value: 0.8963743436923514
    * parameters: {'units_0': 64, 'units_1': 32,  'units_tab_0': 128, 'units_tab_1': 32,
 'activation': 'selu', 'gn': 0.1, 'do': 0.44}


  * Trial 69 with value: 0.8962679154250035
    * parameters: {'units_0': 64, 'units_1': 64, 'units_tab_0': 64, 'units_tab_1': 64,
 'activation': 'gelu', 'gn': 0.05, 'do': 0.32}

##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              'activation': trial.suggest_categorical('activation', ["relu","mish","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

 * Trial 91 with value: 0.902684004334417
    * parameters: {'units_0': 128,
 'units_1': 64,
 'units_tab_0': 256,
 'units_tab_1': 64,
 'activation': 'mish',
 'gn': 0.09,
 'do': 0.2}

#### Fit the model

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
# MODEL 00 'units_0': 64, 'units_1': 64, 'units_tab_0': 64, 'units_tab_1': 64, 'activation': 'gelu', 'gn': 0.05, 'do': 0.32
model= make_model(units=[64,32], units_tab=[128,32], activation="selu", gn=0.10, do=0.44, output_bias=0) # Model v3 (best 0.86323)
model= make_model(units=[64,64], units_tab=[64,64], activation="gelu", gn=0.05, do=0.32, output_bias=0) # Model v3 (best 0.86323)
# MODEL 01
model= make_model(units=[128,64], units_tab=[256,64], activation="mish", gn=0.09, do=0.20, output_bias=0) # Model v3 (best 0.86323)

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv0= make_model(units=[64,64], units_tab=[64,64], activation="gelu", gn=0.05, do=0.32, output_bias=0) # Model v3 (best 0.86323)

resampled_history_v0 = model_cv0.fit(
                              resampled_ds,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation
                              )

model_cv0.evaluate(dataset_validation)

In [ ]:
plot_metrics(resampled_history_v0)

In [ ]:
model_cv1= make_model(units=[128,64], units_tab=[256,64], activation="mish", gn=0.09, do=0.20, output_bias=0, lr=5e-4) # Model v3 (best 0.86323)

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3,  factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
plot_metrics(resampled_history_v1)

#### Evaluate Model:

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

In [ ]:
plt.scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1)/2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_enhanced_v0_all_data_ext.csv")
df_subm

### 4.3 LSTM-Tab with Transformer v1 - Last Work

In [ ]:
inputs.shape

In [ ]:
def make_model_cnn_v4(input_shape, metrics=None, filters=32, kernel_size=3, kr=0.01, sdo=0.2,
                    units_dense=128, gn=0.025, activation="relu", do=0.3, strides=2):
    """
    Improved 1D CNN model - Strided Convolutions instead of MaxPooling.
    """
    if metrics is None:
        metrics = ['accuracy']

    data = keras.layers.Input(shape=input_shape, name="input_layer")
    data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)

    # --- Convolutional Block 1 ---
    cnn_out = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_1")(data_noised)
    cnn_out = keras.layers.LayerNormalization(name="ln_1")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_1")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_1")(cnn_out)
    # NO MaxPooling

    # --- Convolutional Block 2 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 2, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_2")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_2")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_2")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_2")(cnn_out)
    # NO MaxPooling

   # --- Convolutional Block 3 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 4, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_3")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_3")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_3")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_3")(cnn_out)
    # NO MaxPooling

    # --- Pooling and Concatenation ---
    cnn_out_ave = keras.layers.GlobalAveragePooling1D(name="average_pool_final")(cnn_out)
    cnn_out_max = keras.layers.GlobalMaxPooling1D(name="max_pool_final")(cnn_out)
    x = keras.layers.Concatenate(name="concat")([cnn_out_ave, cnn_out_max])

    # --- Dense Layer ---
    x = keras.layers.Dense(units_dense, name="dense_1", kernel_regularizer=keras.regularizers.l2(kr))(x)
    x = keras.layers.BatchNormalization(name="batch_dense")(x)
    x = keras.layers.Activation(activation, name="act_dense")(x)
    x = keras.layers.Dropout(do, name="do_dense")(x)

    # --- Output Layer ---
    outputs = keras.layers.Dense(1, activation='sigmoid', name="output")(x)

    # --- Model Creation and Compilation ---
    model = keras.Model(inputs=data, outputs=outputs, name="cnn_v4")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model

model= make_model_cnn_v4(input_shape=(inputs.shape[1], inputs.shape[2]),strides=2, metrics=METRICS)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v4, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn_v4, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 66 with value: 0.8893146019582802
    * {'kernel_size': 2, 'strides': 1, 'units_dense': 64, 'sdo': 0.43, 'kr': 0.0202, 'activation':'elu', 'gn': 0.07,'do': 0.36}


##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v4, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model_cnn_v4, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 4 with value: 0.0.9343330769888062
    * {'kernel_size': 3, 'strides': 2, 'units_dense': 64, 'sdo': 0.37, 'kr': 0.0040221083422204585, 'activation': 'relu', 'gn': 0.10, 'do': 0.26}


##### Optimization 02

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v4, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v2, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v2,validation_data=dataset_validation_v2, model_class=make_model_cnn_v4, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 4 with value: 0.8672805848378368
    * {'kernel_size': 2, 'strides': 2, 'units_dense': 128, 'sdo': 0.31, 'kr': 0.003562788704053626, 'activation': 'leaky_relu', 'gn': 0.02, 'do': 0.26}


#### Fit the model 00

In [ ]:
model_cv0= make_model_cnn_v4(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 1, 'units_dense': 64, 'sdo': 0.43, 'kr': 0.0202, 'activation':'elu', 'gn': 0.07,'do': 0.36})
model_cv0.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history_v0 = model_cv0.fit(
                              resampled_ds,
                              epochs=2201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001),
                                         keras.callbacks.EarlyStopping(patience=101, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history_v0)

In [ ]:
#plot_model(model, show_shapes=True)

#### Fit the model 01

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv1= make_model_cnn_v4(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 3, 'strides': 2, 'units_dense': 64, 'sdo': 0.37, 'kr': 0.0040221083422204585, 'activation': 'relu', 'gn': 0.10, 'do': 0.26})

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
plot_metrics(resampled_history_v1)

#### Fit the model 02

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv2= make_model_cnn_v4(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 2, 'units_dense': 128, 'sdo': 0.31, 'kr': 0.003562788704053626, 'activation': 'leaky_relu','gn': 0.02,'do': 0.26})

resampled_history_v2 = model_cv2.fit(
                              resampled_ds_v2,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v2
                              )

model_cv2.evaluate(dataset_validation_v2)

In [ ]:
plot_metrics(resampled_history_v2)

#### Evaluate Model:

**MODEL 00**

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

**MODEL 01**

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

**MODEL 02**

In [ ]:
train_predictions_resampled_cv2 = model_cv2.predict(dataset_train_v2)
valid_predictions_resampled_cv2 = model_cv2.predict(dataset_validation_v2)
test_predictions_resampled_cv2 = model_cv2.predict(dataset_test_v2)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)
ax[1].scatter(test_predictions_resampled_cv1,test_predictions_resampled_cv2)
plt.show()

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
plot_cm(y_valid_v1, valid_predictions_resampled_cv1)

In [ ]:
plot_cm(y_valid_v2, valid_predictions_resampled_cv2)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape, test_predictions_resampled_cv2.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1+test_predictions_resampled_cv2)/3
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext.csv")
df_subm

In [ ]:
df_subm["rainfall"] = test_predictions_resampled_cv0
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext_00.csv")
df_subm["rainfall"] = test_predictions_resampled_cv1
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext_01.csv")
df_subm["rainfall"] = test_predictions_resampled_cv2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext_02.csv")

### 4.3 LSTM-Tab with Transformer v2 - Last Work

In [ ]:
inputs.shape

In [ ]:
def make_model_cnn_v5(input_shape, metrics=None, filters=32, kernel_size=3, kr=0.01, sdo=0.2,
                    units_dense=128, gn=0.025, activation="relu", do=0.3, strides=2,
                      gru_units = 64):
    """
    Improved 1D CNN model - Strided Convolutions instead of MaxPooling.
    """
    if metrics is None:
        metrics = ['accuracy']

    data = keras.layers.Input(shape=input_shape, name="input_layer")
    data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
    data_tabular = data_noised[:, 6, :]

    ############################# Convolutional Section #############################
    # --- Convolutional Block 1 ---
    cnn_out = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_1")(data_noised)
    cnn_out = keras.layers.LayerNormalization(name="ln_1")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_1")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_1")(cnn_out)
    # NO MaxPooling

    # --- Convolutional Block 2 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 2, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_2")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_2")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_2")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_2")(cnn_out)
    # NO MaxPooling

   # --- Convolutional Block 3 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 4, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_3")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_3")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_3")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_3")(cnn_out)
    # NO MaxPooling

    # --- Pooling and Concatenation ---
    cnn_out_ave = keras.layers.GlobalAveragePooling1D(name="average_pool_final")(cnn_out)
    cnn_out_max = keras.layers.GlobalMaxPooling1D(name="max_pool_final")(cnn_out)
    x_conv = keras.layers.Concatenate(name="concat")([cnn_out_ave, cnn_out_max])

    ############################# Recursive Section #############################
    whole_seq_output, final_memory_state_00 = keras.layers.GRU(gru_units, return_sequences=True, name="gru_00",return_state=True,recurrent_dropout=do,kernel_regularizer=keras.regularizers.l2(kr))(data_noised)
    whole_seq_output, final_memory_state_01 = keras.layers.GRU(gru_units, return_sequences=True, name="gru_01",return_state=True,recurrent_dropout=do,kernel_regularizer=keras.regularizers.l2(kr))(whole_seq_output)
    whole_seq_output, final_memory_state_10 = keras.layers.GRU(int(gru_units/2), name="gru_1",return_state=True,recurrent_dropout=do,kernel_regularizer=keras.regularizers.l2(kr))(whole_seq_output)

    x_states = keras.layers.Concatenate(name="concat_recursive_states")([final_memory_state_00, final_memory_state_01, final_memory_state_10])
    ############################# Final Dense Section #############################
    # --- Dense Layer ---

    x = keras.layers.Concatenate(name="concat_dense")([x_conv, whole_seq_output,x_states,data_tabular])
    x = keras.layers.Dense(units_dense, name="dense_1", kernel_regularizer=keras.regularizers.l2(kr))(x)
    x = keras.layers.BatchNormalization(name="batch_dense")(x)
    x = keras.layers.Activation(activation, name="act_dense")(x)
    x = keras.layers.Dropout(do, name="do_dense")(x)

    # --- Output Layer ---
    outputs = keras.layers.Dense(1, activation='sigmoid', name="output")(x)

    # --- Model Creation and Compilation ---
    model = keras.Model(inputs=data, outputs=outputs, name="cnn_v4")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=5e-4),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model

model= make_model_cnn_v5(input_shape=(inputs.shape[1], inputs.shape[2]),strides=2, metrics=METRICS)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v5, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'gru_units': trial.suggest_categorical('gru_units', [128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn_v5, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 66 with value: 0.8993543351780899
    * {'kernel_size': 2, 'strides': 2, 'units_dense': 256, 'gru_units': 64, 'sdo': 0.32,
       'kr': 0.0028, 'activation': 'leaky_relu', 'gn': 0.09, 'do': 0.27}


##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v5, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'gru_units': trial.suggest_categorical('gru_units', [128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model_cnn_v5, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 4 with value: 0.9028423172242874.
    * {'kernel_size': 2,  'strides': 2, 'units_dense': 64, 'gru_units': 64, 'sdo': 0.4, 'kr': 0.0282, 'activation': 'elu', 'gn': 0.03,'do': 0.29}


##### Optimization 02

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v5, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'gru_units': trial.suggest_categorical('gru_units', [128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v2, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v2,validation_data=dataset_validation_v2, model_class=make_model_cnn_v5, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 4 with value: 0.8861020235066037
    * {'kernel_size': 2, 'strides': 1, 'units_dense': 128, 'gru_units': 128, 'sdo': 0.23, 'kr': 0.0103, 'activation': 'elu', 'gn': 0.09, 'do': 0.32}


#### Fit the model 00

In [ ]:
model_cv0= make_model_cnn_v5(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 2, 'units_dense': 256, 'gru_units': 64, 'sdo': 0.32, 'kr': 0.0028, 'activation': 'leaky_relu', 'gn': 0.09, 'do': 0.27})
model_cv0.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history_v0 = model_cv0.fit(
                                      resampled_ds,
                                      epochs=2201,
                                      steps_per_epoch=52,
                                      callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001),
                                                keras.callbacks.EarlyStopping(patience=101, restore_best_weights=True, monitor="val_auc",
                                                                              start_from_epoch=3, mode="max")],
                                      validation_data=dataset_validation
                                      )

In [ ]:
model_cv0.save('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_00.keras')
#model_cv0 = keras.models.load_model('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_00.keras')
plot_metrics(resampled_history_v0)

In [ ]:
#plot_model(model, show_shapes=True)

#### Fit the model 01

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv1= make_model_cnn_v5(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 2, 'units_dense': 64, 'gru_units': 64, 'sdo': 0.4, 'kr': 0.0282, 'activation': 'elu', 'gn': 0.03,'do': 0.29})

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
model_cv1.save('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_01.keras')
#model_cv1 = keras.models.load_model('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_01.keras')
plot_metrics(resampled_history_v1)

#### Fit the model 02

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv2= make_model_cnn_v5(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 1, 'units_dense': 128, 'gru_units': 128, 'sdo': 0.23, 'kr': 0.0103, 'activation': 'elu', 'gn': 0.09, 'do': 0.32})

resampled_history_v2 = model_cv2.fit(
                              resampled_ds_v2,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v2
                              )

model_cv2.evaluate(dataset_validation_v2)

In [ ]:
model_cv2.save('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_02.keras')
#model_cv2 = keras.models.load_model('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_02.keras')
plot_metrics(resampled_history_v2)

#### Evaluate Model:

**MODEL 00**

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

**MODEL 01**

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

**MODEL 02**

In [ ]:
train_predictions_resampled_cv2 = model_cv2.predict(dataset_train_v2)
valid_predictions_resampled_cv2 = model_cv2.predict(dataset_validation_v2)
test_predictions_resampled_cv2 = model_cv2.predict(dataset_test_v2)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)
ax[1].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv2)
plt.show()

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
plot_cm(y_valid_v1, valid_predictions_resampled_cv1)

In [ ]:
plot_cm(y_valid_v2, valid_predictions_resampled_cv2)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape, test_predictions_resampled_cv2.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1+test_predictions_resampled_cv2)/3
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_lstm_v0_all_data_ext.csv")
df_subm

In [ ]:
df_subm["rainfall"] = test_predictions_resampled_cv0
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_lstm_v0_all_data_ext_00.csv")
df_subm["rainfall"] = test_predictions_resampled_cv1
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_lstm_v0_all_data_ext_01.csv")
df_subm["rainfall"] = test_predictions_resampled_cv2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_lstm_v0_all_data_ext_02.csv")

### 4.4 CNN-Tab with Transformer v3 - Last Work

In [ ]:
inputs.shape

In [ ]:
def make_model_cnn_v6(input_shape, metrics=None, filters=32, kernel_size=3, kr=0.01, sdo=0.2,
                    units_dense=128, gn=0.025, activation="relu", do=0.3, strides=2):
    """
    Improved 1D CNN model - Strided Convolutions instead of MaxPooling.
    """
    if metrics is None:
        metrics = ['accuracy']

    data = keras.layers.Input(shape=input_shape, name="input_layer")
    data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
    data_tabular = data_noised[:, 6, :]

    ############################# Convolutional Section #############################
    # --- Convolutional Block 1 ---
    cnn_out = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_1")(data_noised)
    cnn_out = keras.layers.LayerNormalization(name="ln_1")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_1")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_1")(cnn_out)
    # MaxPooling
    cnn_out_01 = keras.layers.GlobalMaxPooling1D(name="max_pool_1")(cnn_out)

    # --- Convolutional Block 2 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 2, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_2")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_2")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_2")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_2")(cnn_out)
    # MaxPooling
    cnn_out_02 = keras.layers.GlobalMaxPooling1D(name="max_pool_2")(cnn_out)

   # --- Convolutional Block 3 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 4, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_3")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_3")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_3")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_3")(cnn_out)
    # MaxPooling
    cnn_out_03 = keras.layers.GlobalMaxPooling1D(name="max_pool_3")(cnn_out)

    # --- Pooling and Concatenation ---
    cnn_out_ave = keras.layers.GlobalAveragePooling1D(name="average_pool_final")(cnn_out)
    cnn_out_max = keras.layers.GlobalMaxPooling1D(name="max_pool_final")(cnn_out)
    x_conv = keras.layers.Concatenate(name="concat")([cnn_out_ave, cnn_out_max])

    x = keras.layers.Concatenate(name="concat_dense")([x_conv, cnn_out_03,cnn_out_02,cnn_out_01,data_tabular])
    x = keras.layers.Dense(units_dense, name="dense_1", kernel_regularizer=keras.regularizers.l2(kr))(x)
    x = keras.layers.BatchNormalization(name="batch_dense")(x)
    x = keras.layers.Activation(activation, name="act_dense")(x)
    x = keras.layers.Dropout(do, name="do_dense")(x)

    # --- Output Layer ---
    outputs = keras.layers.Dense(1, activation='sigmoid', name="output")(x)

    # --- Model Creation and Compilation ---
    model = keras.Model(inputs=data, outputs=outputs, name="cnn_v6")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=5e-4),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model

model= make_model_cnn_v6(input_shape=(inputs.shape[1], inputs.shape[2]),strides=2, metrics=METRICS)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v6, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), study_name="resid_cnn")
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn_v6, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 66 with value: 0.8943522066127431
    * {'kernel_size': 3,
 'strides': 1,
 'units_dense': 256,
 'sdo': 0.38,
 'kr': 0.05689652604828707,
 'activation': 'silu',
 'gn': 0.060000000000000005,
 'do': 0.23}


##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v6, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model_cnn_v6, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 4 with value: 0.8825283018867924.
    * {'kernel_size': 3,
 'strides': 2,
 'units_dense': 128,
 'sdo': 0.30000000000000004,
 'kr': 0.02412665758667391,
 'activation': 'elu',
 'gn': 0.03,
 'do': 0.36}


##### Optimization 02

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v6, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v2, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v2,validation_data=dataset_validation_v2, model_class=make_model_cnn_v6, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 4 with value: 0.8991679793206511
    * {'kernel_size': 2,
 'strides': 1,
 'units_dense': 128,
 'sdo': 0.42,
 'kr': 0.048,
 'activation': 'silu',
 'gn': 0.03,
 'do': 0.38}


#### Fit the model 00

In [ ]:
model_cv0= make_model_cnn_v6(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 3, 'strides': 1, 'units_dense': 256, 'sdo': 0.38, 'kr': 0.057, 'activation': 'silu', 'gn': 0.06, 'do': 0.23})
model_cv0.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history_v0 = model_cv0.fit(
                                      resampled_ds,
                                      epochs=2201,
                                      steps_per_epoch=52,
                                      callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001),
                                                keras.callbacks.EarlyStopping(patience=101, restore_best_weights=True, monitor="val_auc",
                                                                              start_from_epoch=3, mode="max")],
                                      validation_data=dataset_validation
                                      )

In [ ]:
model_cv0.save('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_resnet_v0_00.keras')
#model_cv0 = keras.models.load_model('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_00.keras')
plot_metrics(resampled_history_v0)

In [ ]:
#plot_model(model, show_shapes=True)

#### Fit the model 01

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv1= make_model_cnn_v6(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 3, 'strides': 2, 'units_dense': 128, 'sdo': 0.30, 'kr': 0.024, 'activation': 'elu', 'gn': 0.03, 'do': 0.36})

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
model_cv1.save('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_resnet_v0_01.keras')
#model_cv1 = keras.models.load_model('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_01.keras')
plot_metrics(resampled_history_v1)

#### Fit the model 02

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv2= make_model_cnn_v6(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 1, 'units_dense': 128, 'sdo': 0.42, 'kr': 0.048, 'activation': 'silu', 'gn': 0.03, 'do': 0.38})

resampled_history_v2 = model_cv2.fit(
                              resampled_ds_v2,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v2
                              )

model_cv2.evaluate(dataset_validation_v2)

In [ ]:
model_cv2.save('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_resnet_v0_03.keras')
#model_cv2 = keras.models.load_model('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_02.keras')
plot_metrics(resampled_history_v2)

#### Evaluate Model:

**MODEL 00**

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

**MODEL 01**

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

**MODEL 02**

In [ ]:
train_predictions_resampled_cv2 = model_cv2.predict(dataset_train_v2)
valid_predictions_resampled_cv2 = model_cv2.predict(dataset_validation_v2)
test_predictions_resampled_cv2 = model_cv2.predict(dataset_test_v2)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)
ax[1].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv2)
plt.show()

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
plot_cm(y_valid_v1, valid_predictions_resampled_cv1)

In [ ]:
plot_cm(y_valid_v2, valid_predictions_resampled_cv2)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape, test_predictions_resampled_cv2.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1+test_predictions_resampled_cv2)/3
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_resnet_v0_all_data_ext.csv")
df_subm

In [ ]:
df_subm["rainfall"] = test_predictions_resampled_cv0
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_resnet_v0_all_data_ext_00.csv")
df_subm["rainfall"] = test_predictions_resampled_cv1
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_resnet_v0_all_data_ext_01.csv")
df_subm["rainfall"] = test_predictions_resampled_cv2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_resnet_v0_all_data_ext_02.csv")

### 4.5 LSTM-ResNet

In [ ]:
inputs.shape

In [ ]:
def make_model_cnn_v7(input_shape, metrics=None, units=32, kr=0.01, sdo=0.2,
                      units_dense=128, gn=0.025, activation="relu", do=0.3, kr_gru=0.01):
    """
    Improved 1D CNN model - Strided Convolutions instead of MaxPooling.
    """
    if metrics is None:
        metrics = ['accuracy']

    data = keras.layers.Input(shape=input_shape, name="input_layer")
    data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
    data_tabular = data_noised[:, 6, :]

    ############################# RNN Section #############################
    # ------------------ GRU Block 1 ------------------
    gru_out = keras.layers.GRU(units, return_sequences=True, name="gru_1",return_state=False, dropout=sdo,
                               #kernel_regularizer = keras.regularizers.l2(kr_gru)
                               )(data_noised)
    gru_out = keras.layers.LayerNormalization(name="ln_1")(gru_out)
    # ResNet Connection
    resnet_01 = keras.layers.Dense(units,name="dense_resnet_01")(data_noised)
    resnet_01 = keras.layers.Add(name="add_1")([gru_out, resnet_01])

    # Layer for Concat
    resnet_01_flat = keras.layers.Dense(units=1, kernel_regularizer=keras.regularizers.l2(kr),name="dense_flat_resnet_01")(resnet_01)
    resnet_01_flat = keras.layers.Reshape((7,), name="flat_resnet_01")(resnet_01_flat)
    resnet_01_flat = keras.layers.BatchNormalization(name="batch_flat_resnet_01")(resnet_01_flat)
    resnet_01_flat = keras.layers.Activation(activation, name="act_flat_resnet_01")(resnet_01_flat)


    # ------------------ GRU Block 2 ------------------
    gru_out = keras.layers.GRU(units*2, return_sequences=True, name="gru_2",return_state=False, dropout=sdo,
                               #kernel_regularizer = keras.regularizers.l2(kr_gru)
                               )(resnet_01)
    gru_out = keras.layers.LayerNormalization(name="ln_2")(gru_out)
    # ResNet Connection
    resnet_02 = keras.layers.Dense(units*2,name="dense_resnet_02")(resnet_01)
    resnet_02 = keras.layers.Add(name="add_2")([gru_out, resnet_02])

    # Layer for Concat
    resnet_02_flat = keras.layers.Dense(units=1, kernel_regularizer=keras.regularizers.l2(kr),name="dense_flat_resnet_02")(resnet_02)
    resnet_02_flat = keras.layers.Reshape((7,),name="flat_resnet_02")(resnet_02_flat)
    resnet_02_flat = keras.layers.BatchNormalization(name="batch_flat_resnet_02")(resnet_02_flat)
    resnet_02_flat = keras.layers.Activation(activation, name="act_flat_resnet_02")(resnet_02_flat)


    # ------------------ GRU Block 3 ------------------
    gru_out = keras.layers.GRU(units*3, return_sequences=False, name="gru_3",return_state=False, dropout=sdo,
                               kernel_regularizer = keras.regularizers.l2(kr_gru)
                               )(resnet_02)
    gru_out = keras.layers.LayerNormalization(name="ln_3")(gru_out)


    # --- Pooling and Concatenation ---
    x = keras.layers.Concatenate(name="concat_dense")([resnet_01_flat, resnet_02_flat,gru_out,data_tabular])
    x = keras.layers.Dense(units_dense, name="dense_1", kernel_regularizer=keras.regularizers.l2(kr))(x)
    x = keras.layers.BatchNormalization(name="batch_dense")(x)
    x = keras.layers.Activation(activation, name="act_dense")(x)
    x = keras.layers.Dropout(do, name="do_dense")(x)

    # --- Output Layer ---
    outputs = keras.layers.Dense(1, activation='sigmoid', name="output")(x)

    # --- Model Creation and Compilation ---
    model = keras.Model(inputs=data, outputs=outputs, name="cnn_v4")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=5e-4),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model

model= make_model_cnn_v7(input_shape=(inputs.shape[1], inputs.shape[2]),metrics=METRICS)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

In [ ]:
# for a,b in resampled_ds.take(1):
#   print(a.shape,b.shape)

# model.predict(a)

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v7, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'units': trial.suggest_categorical('units', [96, 64, 32]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'kr_gru': trial.suggest_float('kr_gru', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }


    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    try:
      # Fit the model
      model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                        keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                      start_from_epoch=3, mode="max")]
              )

      # Make predictions on the validation set
      y_pred = model.predict(validation_data)

      # Calculate the RMSE for the current fold
      rmse_scores = roc_auc_score(y_valid, y_pred)

    except ValueError as e:  # or any other relevant exception type
      # Handle the case where the model is invalid, e.g., due to shape issues
      print(f"Trial failed due to ValueError: {e}")
      print(f"Hyperparameters: {trial.params}")  # Log the parameters for debugging
      rmse_scores = float('inf')  # or any appropriate penalty value

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn_v7, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

Best Score: 0.9018731375053213

{'units_dense': 256,
 'units': 64,
 'sdo': 0.44,
 'kr': 0.0019,
 'kr_gru': 0.017,
 'activation': 'leaky_relu',
 'gn': 0.03,
 'do': 0.37}

##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v7, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'units': trial.suggest_categorical('units', [96, 64, 32]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'kr_gru': trial.suggest_float('kr_gru', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model_cnn_v7, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

Best: 0.9119191919191919

{'units_dense': 256,
 'units': 96,
 'sdo': 0.22,
 'kr': 0.0022,
 'kr_gru': 0.0381,
 'activation': 'elu',
 'gn': 0.08,
 'do': 0.4}

##### Optimization 02

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v7, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'units': trial.suggest_categorical('units', [96, 64, 32]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'kr_gru': trial.suggest_float('kr_gru', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v2, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v2,validation_data=dataset_validation_v2, model_class=make_model_cnn_v7, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

Best:0.8685124601155134.

{'units_dense': 256,
 'units': 96,
 'sdo': 0.28,
 'kr': 0.06505678914953572,
 'kr_gru': 0.026089087169453223,
 'activation': 'gelu',
 'gn': 0.04,
 'do': 0.33}

#### Fit the model 00

In [ ]:
model_cv0= make_model_cnn_v7(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'units_dense': 256,
                                                                                                 'units': 64,
                                                                                                 'sdo': 0.44,
                                                                                                 'kr': 0.0019,
                                                                                                 'kr_gru': 0.017,
                                                                                                 'activation': 'leaky_relu',
                                                                                                 'gn': 0.03,
                                                                                                 'do': 0.37})
model_cv0.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history_v0 = model_cv0.fit(
                                      resampled_ds,
                                      epochs=2201,
                                      steps_per_epoch=52,
                                      callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001),
                                                keras.callbacks.EarlyStopping(patience=101, restore_best_weights=True, monitor="val_auc",
                                                                              start_from_epoch=3, mode="max")],
                                      validation_data=dataset_validation
                                      )

In [ ]:
model_cv0.save('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/lstm_resnet_v0_00.keras')
#model_cv0 = keras.models.load_model('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_00.keras')
plot_metrics(resampled_history_v0)

In [ ]:
#plot_model(model, show_shapes=True)

#### Fit the model 01

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv1= make_model_cnn_v7(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'units_dense': 256,
                                                                                                 'units': 96,
                                                                                                 'sdo': 0.22,
                                                                                                 'kr': 0.0022,
                                                                                                 'kr_gru': 0.0381,
                                                                                                 'activation': 'elu',
                                                                                                 'gn': 0.08,
                                                                                                 'do': 0.4})

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
model_cv1.save('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/lstm_resnet_v0_01.keras')
#model_cv1 = keras.models.load_model('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_01.keras')
plot_metrics(resampled_history_v1)

#### Fit the model 02

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv2= make_model_cnn_v7(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'units_dense': 256, 'units': 96, 'sdo': 0.28, 'kr': 0.0651, 'kr_gru': 0.0261, 'activation': 'gelu', 'gn': 0.04, 'do': 0.33})

resampled_history_v2 = model_cv2.fit(
                              resampled_ds_v2,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v2
                              )

model_cv2.evaluate(dataset_validation_v2)

In [ ]:
model_cv2.save('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/lstm_resnet_v0_02.keras')
#model_cv2 = keras.models.load_model('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/cnn_lstm_v2_02.keras')
plot_metrics(resampled_history_v2)

#### Evaluate Model:

**MODEL 00**

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

**MODEL 01**

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

**MODEL 02**

In [ ]:
train_predictions_resampled_cv2 = model_cv2.predict(dataset_train_v2)
valid_predictions_resampled_cv2 = model_cv2.predict(dataset_validation_v2)
test_predictions_resampled_cv2 = model_cv2.predict(dataset_test_v2)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)
ax[1].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv2)
plt.show()

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
plot_cm(y_valid_v1, valid_predictions_resampled_cv1)

In [ ]:
plot_cm(y_valid_v2, valid_predictions_resampled_cv2)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape, test_predictions_resampled_cv2.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1+test_predictions_resampled_cv2)/3
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_resnet_v0_all_data_ext.csv")
df_subm

In [ ]:
df_subm["rainfall"] = test_predictions_resampled_cv0
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_resnet_v0_all_data_ext_00.csv")
df_subm["rainfall"] = test_predictions_resampled_cv1
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_resnet_v0_all_data_ext_01.csv")
df_subm["rainfall"] = test_predictions_resampled_cv2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_resnet_v0_all_data_ext_02.csv")

### 4.3 CNN Classifier

In [ ]:
def make_model_cnn(metrics=METRICS, filters=64, kernel_size=2, kr=0.01, sdo=0.2,
               units_dense=256, output_bias=None, gn=0.025, activation="relu", do=0.3):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")

  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  # Cnn Layers
  # Layer 0
  cnn_out = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=1,
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_0")(data_noised)
  cnn_out = keras.layers.LayerNormalization(name="ln_0")(cnn_out)
  cnn_out = keras.layers.Activation("relu", name="act_0")(cnn_out)
  cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_0")(cnn_out)
  cnn_out = keras.layers.MaxPooling1D(pool_size=2, name="max_pool_1")(cnn_out)
  # Layer 1
  cnn_out = keras.layers.Conv1D(filters=int(filters*2), kernel_size=kernel_size, strides=1,
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_1")(cnn_out)
  cnn_out = keras.layers.LayerNormalization(name="ln_1")(cnn_out)
  cnn_out = keras.layers.Activation("relu", name="act_1")(cnn_out)
  cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_1")(cnn_out)

  #outputs
  cnn_out_ave = keras.layers.GlobalAveragePooling1D(name="average_pool_final")(cnn_out)
  cnn_out_max = keras.layers.GlobalMaxPooling1D(name="max_pool_final")(cnn_out)

  x = keras.layers.Concatenate(name="concat")([cnn_out_ave, cnn_out_max])

  # Dense Layer
  x = keras.layers.Dense(units_dense, name="dense_1")(x)
  x = keras.layers.BatchNormalization(name="batch_dense")(x)
  x = keras.layers.Activation(activation, name="act_dense")(x)
  x = keras.layers.Dropout(do, name="do_dense")(x)

  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="cnn_v0")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=5e-4),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model_cnn(output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn, use_gpu=False, rs=42, fit_scaling=False, epochs=21):

    model_class = model

    params = {
              'filters': trial.suggest_categorical('filters', [16,32,64]),
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),
              'kr': trial.suggest_float('kr', 0.01, 0.1, step=0.01),
              'activation': trial.suggest_categorical('activation', ["relu","selu","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=9, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=make_model_cnn, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

*  Best is trial 66 with value: 0.895310061
    * {'filters': 64,
 'kernel_size': 2,
 'units_dense': 256,
 'do': 0.39,
 'sdo': 0.39,
 'kr': 0.09999999999999999,
 'activation': 'gelu',
 'gn': 0.09999999999999999}


#### Fit the model

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
model= make_model_cnn(**{'filters': 64, 'kernel_size': 2, 'units_dense': 256, 'do': 0.39, 'sdo': 0.39, 'kr': 0.1, 'activation': 'gelu', 'gn': 0.1}) # Model v

model.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history = model.fit(
                              resampled_ds,
                              epochs=2201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001),
                                         keras.callbacks.EarlyStopping(patience=101, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history)

#### Evaluate Model:

In [ ]:
train_predictions_resampled = model.predict(dataset_train)
valid_predictions_resampled = model.predict(dataset_validation)
test_predictions_resampled = model.predict(dataset_test)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
df_subm["rainfall"] = test_predictions_resampled
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v2_all_data_ext.csv")

### 4.4 CNN Classifier - Tab v0

In [ ]:
def make_model_cnn(metrics=METRICS, filters=64, kernel_size=2, kr=0.01, sdo=0.2,
                   units_dense=256, output_bias=None, gn=0.025, activation="relu",
                   do=0.3, units_tab=[128,128], lr=5e-4):

  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  data_tabular = data[:, 6, :]

  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  # Cnn Layers
  # Layer 0
  cnn_out = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=1,
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_0")(data_noised)
  cnn_out = keras.layers.LayerNormalization(name="ln_cnn_0")(cnn_out)
  cnn_out = keras.layers.Activation("relu", name="act_cnn_0")(cnn_out)
  cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_cnn_0")(cnn_out)
  cnn_out = keras.layers.MaxPooling1D(pool_size=2, name="max_pool_cnn_1")(cnn_out)
  # Layer 1
  cnn_out = keras.layers.Conv1D(filters=int(filters*2), kernel_size=kernel_size, strides=1,
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_1")(cnn_out)
  cnn_out = keras.layers.LayerNormalization(name="ln_cnn_1")(cnn_out)
  cnn_out = keras.layers.Activation("relu", name="act_cnn_1")(cnn_out)
  cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_cnn_1")(cnn_out)

  #outputs
  cnn_out_ave = keras.layers.GlobalAveragePooling1D(name="average_pool_final")(cnn_out)
  cnn_out_max = keras.layers.GlobalMaxPooling1D(name="max_pool_final")(cnn_out)

  x = keras.layers.Concatenate(name="concat")([cnn_out_ave, cnn_out_max])

  # Dense Layer
  x = keras.layers.Dense(units_dense, name="dense_conv_1")(x)
  x = keras.layers.BatchNormalization(name="batch_dense_conv")(x)
  x = keras.layers.Activation(activation, name="act_dense_conv")(x)
  x = keras.layers.Dropout(do, name="do_dense_conv")(x)

  # Tabular Section
  tabx = keras.layers.Dense(units_tab[0], name="dense_0")(data_tabular)
  tabx = keras.layers.BatchNormalization(name="batch_0")(tabx)
  tabx = keras.layers.Activation(activation, name="act_0")(tabx)
  tabx = keras.layers.Dropout(do, name="do_0")(tabx)
  tabx = keras.layers.Dense(units_tab[1], name="dense_1")(tabx)
  tabx = keras.layers.BatchNormalization(name="batch_1")(tabx)
  tabx = keras.layers.Activation(activation, name="act_1")(tabx)
  tabx = keras.layers.Dropout(do, name="do_1")(tabx)

  # Final Concatenation
  x = keras.layers.Concatenate(name="final_concat")([x, tabx])

  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="cnn_v0")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=lr),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model_cnn(output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn, use_gpu=False, rs=42, fit_scaling=False,epochs=51):

    model_class = model

    params = {
              'filters': trial.suggest_categorical('filters', [32,64,128]),
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),
              'kr': trial.suggest_float('kr', 0.01, 0.1, step=0.01),
              'activation': trial.suggest_categorical('activation', ["relu","selu","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01),
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn, n_trials=151, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

 * Best is trial 119 with value: 0.9003476656733361.
    * parameters: {'filters': 32,
 'kernel_size': 3,
 'units_dense': 256,
 'do': 0.26,
 'sdo': 0.26,
 'kr': 0.08,
 'activation': 'relu',
 'gn': 0.03,
 'units_tab_0': 128,
 'units_tab_1': 64}.


 * Best is trial 129 with value: 0.897686958989641.
    * parameters: {'filters': 64, 'kernel_size': 3, 'units_dense': 64, 'sdo': 0.22, 'kr': 0.05, 'activation': 'selu', 'gn': 0.06, 'do': 0.2, 'units_tab_0': 128, 'units_tab_1': 64}


#### Fit the model

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
model= make_model_cnn(lr=5e-4, **{'filters': 32, 'kernel_size': 3, 'units_dense': 256, 'do': 0.26, 'sdo': 0.26, 'kr': 0.08, 'activation': 'relu', 'gn': 0.03, "units_tab":[128, 64]}) # auc: 0.8876
model= make_model_cnn(lr=5e-4, **{'filters': 64, 'kernel_size': 3, 'units_dense': 64, 'do': 0.20, 'sdo': 0.22, 'kr': 0.05, 'activation': 'relu', 'gn': 0.06, "units_tab":[128, 64]}) # auc: 0.8853


model.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history = model.fit(
                              resampled_ds,
                              epochs=151,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.00001),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history)

#### Evaluate Model:

In [ ]:
train_predictions_resampled = model.predict(dataset_train)
valid_predictions_resampled = model.predict(dataset_validation)
test_predictions_resampled = model.predict(dataset_test)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
df_subm["rainfall"] = test_predictions_resampled
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_tab_v1_all_data_ext.csv")

### 4.5 Temporal Transformer

In [ ]:
num_samples = 5000
sequence_length = 7
num_features = 30
embed_dim = num_features
head_size = 32
num_heads = 4
gru_units = 64
dropout = 0.1

# 2. Positional Embedding Layer
class TimeSeriesPositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

# 3. Transformer Encoder Layer with GRU
def transformer_encoder_gru(inputs, head_size, num_heads, gru_units, embed_dim, dropout=0):
    # Attention and Normalization
    x = keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part (GRU)
    x = keras.layers.GRU(units=gru_units, return_sequences=True)(res)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.Dense(units=embed_dim)(x) # Added Dense Layer
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [ ]:
def make_model(metrics=METRICS, units=[32,32], output_bias=None, gn=0.025, activation="relu", do=0.3, num_transformer_layers=2,
               head_size=32, num_heads=4, gru_units=64):
  """
  Builds a transformer-based binary classification model for time series data.

  Args:
      input_shape: Tuple, the shape of the input data (sequence_length, num_features).
      num_transformer_layers: Integer, the number of transformer encoder layers to stack.
      head_size: Integer, the size of each attention head.
      num_heads: Integer, the number of attention heads.
      gru_units: Integer, the number of GRU units in the feed-forward part.
      dropout: Float, the dropout rate.

  Returns:
      A Keras model.
  """

  tot_obs, sequence_length, num_features = inputs.shape
  embed_dim = inputs.shape[2]

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  data_tabular = data[:, 6, :]
  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  lstm_out = keras.layers.LSTM(units[0], return_sequences=True, name="lstm_0",)(data_noised)
  lstm_out = keras.layers.LSTM(units[1], name="lstm_1")(lstm_out)
  lstm_out = keras.layers.Dropout(do)(lstm_out)

  # Tabular Section

  x = TimeSeriesPositionalEmbedding(sequence_length, embed_dim)(data)

  # Stack Transformer Layers
  for _ in range(num_transformer_layers):
      x = transformer_encoder_gru(x, head_size, num_heads, gru_units, embed_dim, dropout)

  # Global Pooling and Output Layer
  x_av = keras.layers.GlobalAveragePooling1D()(x)
  x_av = keras.layers.Dropout(do)(x_av)
  # Max Pooling and Output Layer
  x_mx = keras.layers.GlobalMaxPooling1D()(x)
  x_mx = keras.layers.Dropout(do)(x_mx)


  # Concatenate
  x = keras.layers.Concatenate(name="concat")([lstm_out, x_av,x_mx])

  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="temp_trans_v0")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model(output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

In [ ]:
#plot_model(model, show_shapes=True)

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'output_bias':0,
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.15, 0.45, step=0.01),
              'num_transformer_layers': trial.suggest_categorical('num_transformer_layers', [2,3]),
              'head_size': trial.suggest_categorical('head_size', [32,64]),
              'num_heads': trial.suggest_categorical('num_heads', [4,8,16]),
              'gru_units': trial.suggest_categorical('gru_units', [32,64,128])
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, verbose=1, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

* Best is trial 91 with value: 0.8922236412657868
  * Parameters: {'units_0': 128, 'units_1': 64, 'gn': 0.08, 'do': 0.43999999999999995, 'num_transformer_layers': 3, 'head_size': 32, 'num_heads': 4, 'gru_units': 128}

##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'output_bias':0,
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.15, 0.45, step=0.01),
              'num_transformer_layers': trial.suggest_categorical('num_transformer_layers', [2,3]),
              'head_size': trial.suggest_categorical('head_size', [32,64]),
              'num_heads': trial.suggest_categorical('num_heads', [4,8,16]),
              'gru_units': trial.suggest_categorical('gru_units', [32,64,128])
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, verbose=1, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model, n_trials=111, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

#### Fit the model

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
# MODEL 00 'units_0': 64, 'units_1': 64, 'units_tab_0': 64, 'units_tab_1': 64, 'activation': 'gelu', 'gn': 0.05, 'do': 0.32
model= make_model(units=[64,32], units_tab=[128,32], activation="selu", gn=0.10, do=0.44, output_bias=0) # Model v3 (best 0.86323)
model= make_model(units=[64,64], units_tab=[64,64], activation="gelu", gn=0.05, do=0.32, output_bias=0) # Model v3 (best 0.86323)
# MODEL 01
model= make_model(units=[128,64], units_tab=[256,64], activation="mish", gn=0.09, do=0.20, output_bias=0) # Model v3 (best 0.86323)

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

#{'units_0': 128, 'units_1': 64, 'gn': 0.08, 'do': 0.43999999999999995, 'num_transformer_layers': 3, 'head_size': 32, 'num_heads': 4, 'gru_units': 128}

##### Model 00

In [ ]:
model_cv0= make_model(metrics=METRICS, units=[128,64], output_bias=0, gn=0.08, do=0.44, num_transformer_layers=3,
                        head_size=32, num_heads=4, gru_units=128) # Model v0

resampled_history_v0 = model_cv0.fit(
                              resampled_ds,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation
                              )

model_cv0.evaluate(dataset_validation)

In [ ]:
plot_metrics(resampled_history_v0)

##### Model 01

In [ ]:

             epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
plot_metrics(resampled_history_v1)

#### Evaluate Model:

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

In [ ]:
# df_subm["rainfall"] = test_predictions_resampled_cv0
# df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v0_all_data_ext.csv")
df_subm_v0 = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v0_all_data_ext.csv", index_col=0)
df_subm_v0

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

In [ ]:
df_subm_v1 = df_subm.copy()
df_subm_v1["rainfall"] = test_predictions_resampled_cv1
df_subm_v1.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v1_all_data_ext.csv")
df_subm_v1 = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v1_all_data_ext.csv", index_col=0)
df_subm_v1

In [ ]:
plt.scatter(df_subm_v0,df_subm_v1)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape

In [ ]:
df_subm["rainfall"] = (df_subm_v1+df_subm_v0)/2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v_ensemble_all_data_ext.csv")
df_subm

### **5.1 TREE BASED MODELS**

In [ ]:
all_train = pd.concat([validation, train], axis=0)
all_train.shape

In [ ]:
X = all_train.drop("rainfall", axis=1)
y = all_train["rainfall"]
#x.describe().T

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights = dict(enumerate(class_weights))

sample_pos_weight = class_weights[1]/class_weights[0]
sample_pos_weight

In [ ]:
from imblearn.over_sampling import SMOTE

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [ ]:
preprocessing = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                                ('scaler', StandardScaler())])

X_train_res = preprocessing.fit_transform(X_train_res)
X_val = preprocessing.transform(X_val)

In [ ]:
X_train_res.shape, X_val.shape

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf_verify = GaussianNB()
cross_val_score(clf_verify, X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()

#### Random Forest:

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
def objective_rf(trial):
    # Suggest important hyperparameters for RandomForest
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    max_depth = trial.suggest_int("max_depth", 3, 20)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])

    clf = RandomForestClassifier(
        class_weight='balanced',
        random_state=42,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features
    )

    score = cross_val_score(clf,X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()
    return score

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_rf, n_trials=50) #adjust to 50
best_params_rf = study_rf.best_trial.params
print("Best params for RandomForest:", best_params_rf)

* Best Params: {'n_estimators': 304, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}
* Score: 0.9533470932991909

In [ ]:
import optuna.visualization as vis
from IPython.display import IFrame, display

# Plot the optimization history (objective value over
vis.plot_optimization_history(study_rf)
#fig_history.write_html("optimization_history.html")

In [ ]:
# Plot the parameter importances
plot_param_importances(study_rf)
#fig_importances.write_html("parameter_importances.html")

#### GradientBoostingClassifier

In [ ]:
def objective_gb(trial):
    # Suggest key hyperparameters for GradientBoostingClassifier
    n_estimators = trial.suggest_int("n_estimators", 400, 650) #trial.suggest_int("n_estimators", 50, 500)
    learning_rate = trial.suggest_float("learning_rate", 0.05, 0.25, log=True)
    max_features =trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    max_depth = trial.suggest_int("max_depth", 7, 12)
    subsample = trial.suggest_float("subsample", 0.6, 0.95, step=0.025)

    clf = GradientBoostingClassifier(
        random_state=42,
        n_estimators=n_estimators,
        max_features=max_features,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    )

    score = cross_val_score(clf,X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()
    return score

study_gb = optuna.create_study(direction="maximize")
study_gb.optimize(objective_gb, n_trials=30) #adjust to 50
best_params_gb = study_gb.best_trial.params
print("Best params for GradientBoosting:", best_params_gb)

* Best Params: {'n_estimators': 455, 'learning_rate': 0.1516395931910056, 'max_features': 'sqrt', 'max_depth': 9, 'subsample': 0.75}
* Score: 0.9678502444785376

In [ ]:
# Plot the optimization history (objective value over
vis.plot_optimization_history(study_gb)
#fig_history.write_html("optimization_history.html")

In [ ]:
# Plot the parameter importances
plot_param_importances(study_gb)
#fig_importances.write_html("parameter_importances.html")

#### XGBClassifier

In [ ]:
def objective_xgb(trial):
    # Suggest key hyperparameters for XGBClassifier
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    subsample = trial.suggest_float("subsample", 0.5, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 1)
    reg_alpha = trial.suggest_float('reg_alpha', 0.00001, 1.0, log=True)
    reg_lambda = trial.suggest_float('reg_lambda', 0.00001, 1.0, log=True)

    clf = XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda
    )

    score = cross_val_score(clf, X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()
    return score
study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_xgb, n_trials=71) #adjust to 50
best_params_xgb = study_xgb.best_trial.params
print("Best params for XGBoost:", best_params_xgb)

* Parameters: {'n_estimators': 325, 'learning_rate': 0.06278279256080053, 'max_depth': 8, 'subsample': 0.70, 'colsample_bytree': 0.87, 'gamma': 0.17, 'reg_alpha': 0.024, 'reg_lambda': 8.111e-05}
* Score 0.9618260284102824

In [ ]:
# Plot the optimization history (objective value over
vis.plot_optimization_history(study_xgb)
#fig_history.write_html("optimization_history.html")

In [ ]:
# Plot the parameter importances
plot_param_importances(study_xgb)
#fig_importances.write_html("parameter_importances.html")

#### Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

# Build final models using the best parameters from Optuna studies
rf_final = RandomForestClassifier(class_weight='balanced', random_state=42, **best_params_rf)
gb_final = GradientBoostingClassifier(random_state=42, **best_params_gb)
xgb_final = XGBClassifier(use_label_encoder=False, eval_metric="auc", random_state=42, **best_params_xgb)

# Create a VotingClassifier with the best estimators
voting_clf = VotingClassifier(
    estimators=[
        ('rf', rf_final),
        ('gb', gb_final),
        ('xgb', xgb_final)
    ],
    voting='soft'  # or 'soft' if probability estimates are preferred
)

# Fit the VotingClassifier on your resampled training data
voting_clf.fit(X_train_res, y_train_res)

# Evaluate using cross-validation with ROC AUC as the metric.
# If it's a binary classification, use 'roc_auc'.
# For multi-class problems, consider using 'roc_auc_ovr' or 'roc_auc_ovo'
from sklearn.model_selection import cross_val_score
voting_roc_auc = cross_val_score(voting_clf, X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()
print("Voting Classifier CV ROC AUC Score:", voting_roc_auc)

In [ ]:
obj = sio.dump(voting_clf, "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/voting_v0.skops")

unknown_types = sio.get_untrusted_types(file="/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/voting_v0.skops")
# investigate the contents of unknown_types, and only load if you trust
# everything you see.
voting_clf = sio.load("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/voting_v0.skops", trusted=unknown_types)

In [ ]:
y_val_pred_proba = voting_clf.predict_proba(X_val)[:, 1]
print('Validation ROC AUC:', roc_auc_score(y_val, y_val_pred_proba),'\n')
print(classification_report(y_val, voting_clf.predict(X_val)))

In [ ]:
X_test = preprocessing.transform(test)
test_predictions_proba = voting_clf.predict_proba(X_test)[:, 1]

In [ ]:
df_subm["rainfall"] = test_predictions_proba
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_voting_v0_all_data_ext.csv")
df_subm

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)
ax[1].scatter(test_predictions_resampled_cv2,test_predictions_proba)
plt.show()

### **5.1 TREE BASED MODELS - SKTIME**

In [ ]:
%%capture
!pip install sktime

In [ ]:
dataset_test_v2 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_test,
                                                                    y_test,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10000,
                                                                    shuffle=False
                                                                 )


for batch in dataset_test_v2.take(1000):
    test_v2, test_targets_v2 = batch

test_v2 = test_v2.numpy()
test_targets_v2 = test_targets_v2.numpy()

print("Input shape:", test_v2.shape)
print("Target shape:", test_targets_v2.shape)

In [ ]:
from sktime.classification.deep_learning import GRUClassifier
from sktime.classification.interval_based import TimeSeriesForestClassifier

In [ ]:
#all_train = pd.concat([validation, train], axis=0)
all_train.shape, all_target.shape, inputs_v2.shape

In [ ]:
X = all_train.copy()
y = all_target.copy()
X_test = test_v2.copy()
#x.describe().T
X_test.shape

In [ ]:
# from sklearn.utils.class_weight import compute_class_weight

# class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
# class_weights = dict(enumerate(class_weights))

# sample_pos_weight = class_weights[1]/class_weights[0]
# sample_pos_weight

In [ ]:
from imblearn.over_sampling import SMOTE

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
X_train_ = X_train.reshape(X_train.shape[0], -1)

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_, y_train)

X_train_res = X_train_res.reshape(X_train_res.shape[0], X_train.shape[1], X_train.shape[2])
X_train_res.shape

In [ ]:
# preprocessing = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
#                                 ('scaler', StandardScaler())])

# X_train_res = preprocessing.fit_transform(X_train_res)
# X_val = preprocessing.transform(X_val)

In [ ]:
X_train_res.shape, X_val.shape

In [ ]:
# from sklearn.naive_bayes import GaussianNB

# clf_verify = GaussianNB()
# cross_val_score(clf_verify, X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()

#### GRUClassifier:

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
def objective_rf(trial):
    # Suggest important hyperparameters for RandomForest
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256])
    n_layers = trial.suggest_categorical("n_layers", [2, 3, 4, 5])
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.025)
    fc_dropout = trial.suggest_float("fc_dropout", 0.1, 0.5, step=0.025)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])

    clf = GRUClassifier(
        random_state=42,
        hidden_dim=hidden_dim,
        n_layers=n_layers,
        dropout=dropout,
        fc_dropout=fc_dropout,
        bidirectional=bidirectional,
        num_epochs=15,
        batch_size=64,
        verbose=False
    )

    score = cross_val_score(clf,X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()
    return score

study_gru = optuna.create_study(direction="maximize")
study_gru.optimize(objective_rf, n_trials=50) #adjust to 50
best_params_gru = study_gru.best_trial.params
print("Best params for GRUFCNNClassifier:", best_params_gru)

* Best Params: {'n_estimators': 304, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}
* Score: 0.9533470932991909

In [ ]:
import optuna.visualization as vis
from IPython.display import IFrame, display

# Plot the optimization history (objective value over
vis.plot_optimization_history(study_rf)
#fig_history.write_html("optimization_history.html")

In [ ]:
# Plot the parameter importances
plot_param_importances(study_rf)
#fig_importances.write_html("parameter_importances.html")

#### HIVECOTEV2

In [ ]:
from sktime.classification.kernel_based import RocketClassifier
from sktime.classification.ensemble import ComposableTimeSeriesForestClassifier
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.hybrid import HIVECOTEV2

In [ ]:
X_train_res.shape

In [ ]:
def objective_hc2(trial):
    # --- Hyperparameters for component classifiers ---
    drcif_n_estimators = trial.suggest_categorical("drcif_n_estimators", [50, 100, 150, 200])
    arsenal_n_estimators = trial.suggest_categorical("arsenal_n_estimators", [50, 100, 150, 200])
    tde_n_estimators = trial.suggest_categorical("tde_n_estimators", [50, 100, 150, 200])

    # ShapeletTransformClassifier hyperparameters
    stc_n_shapelet_samples = trial.suggest_int("stc_n_shapelet_samples", 1000, 20000, log=True)
    stc_max_shapelet_length_prop = trial.suggest_float("stc_max_shapelet_length_prop", 0.2, 0.8)
    stc_max_shapelets_prop = trial.suggest_float("stc_max_shapelets_prop", 0.01, 0.2)
    stc_max_shapelets = int(stc_n_shapelet_samples * stc_max_shapelets_prop)
    stc_max_shapelet_length = None

    # TemporalDictionaryEnsemble (TDE) Hyperparameters - Corrected
    tde_n_parameter_samples = trial.suggest_int("tde_n_parameter_samples", 50, 250)  # Number of parameter sets to sample
    tde_max_ensemble_size = trial.suggest_int("tde_max_ensemble_size", 25, 100)      # Max size of the ensemble
    tde_max_win_len_prop = trial.suggest_float("tde_max_win_len_prop", 0.5, 1.0)      # Max window length proportion

    # Create the HIVECOTEV2 classifier with tuned parameters
    clf = HIVECOTEV2(
        stc_params={
            "n_shapelet_samples": stc_n_shapelet_samples,
            "max_shapelets": stc_max_shapelets,
            "max_shapelet_length": stc_max_shapelet_length
        },
        drcif_params={"n_estimators": drcif_n_estimators},
        arsenal_params={"n_estimators": arsenal_n_estimators},
        tde_params={  # Corrected TDE parameters
            "n_parameter_samples": tde_n_parameter_samples,
            "max_ensemble_size": tde_max_ensemble_size,
            "max_win_len_prop": tde_max_win_len_prop
        },
        time_limit_in_minutes=0.5,
        random_state=42,
    )

    # Custom scoring function
    def custom_roc_auc(estimator, X, y):
        try:
            if len(np.unique(y)) > 2:
                y_pred_proba = estimator.predict_proba(X)
                return roc_auc_score(y, y_pred_proba, multi_class='ovr')
            else:
                y_pred_proba = estimator.predict_proba(X)[:, 1]
                return roc_auc_score(y, y_pred_proba)
        except AttributeError:
            y_pred = estimator.predict(X)
            return roc_auc_score(y, y_pred)

    y_train_str = y_train.astype(str)
    score = cross_val_score(clf, all_train, all_target, cv=5, scoring=custom_roc_auc).mean()
    return score

# --- Optuna Optimization ---
study_hc2 = optuna.create_study(direction="maximize")
study_hc2.optimize(objective_hc2, n_trials=31)
best_params_hc2 = study_hc2.best_trial.params
print("Best params for HIVE-COTE v2:", best_params_hc2)

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_unit_test
X_train, y_train = load_unit_test(return_X_y=True, split="train")
X_test, y_test = load_unit_test(return_X_y=True, split="test")
classifier = KNeighborsTimeSeriesClassifier(distance="euclidean")
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
X_train.shape

#### KNeighborsTimeSeriesClassifier

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [ ]:
X.shape, y.shape

In [ ]:
X_ = X.reshape(X.shape[0], -1)

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_, y)

X_train_res = X_train_res.reshape(X_train_res.shape[0], X.shape[1], X.shape[2])
X_train_res.shape

In [ ]:
def objective_hc2(trial):
    # --- Hyperparameters for component classifiers ---
    n_neighbors = trial.suggest_categorical("n_neighbors", [3, 4, 5, 6, 7])
    weights = trial.suggest_categorical("weights", ["uniform", "distance"])
    leaf_size = trial.suggest_categorical("leaf_size", [25,30,35,40,50])


    # Create the HIVECOTEV2 classifier with tuned parameters
    clf = KNeighborsTimeSeriesClassifier(n_neighbors=n_neighbors,distance="euclidean",algorithm="ball_tree",leaf_size=leaf_size,weights=weights)

    # Custom scoring function
    def custom_roc_auc(estimator, X, y):
        try:
            if len(np.unique(y)) > 2:
                y_pred_proba = estimator.predict_proba(X)
                return roc_auc_score(y, y_pred_proba, multi_class='ovr')
            else:
                y_pred_proba = estimator.predict_proba(X)[:, 1]
                return roc_auc_score(y, y_pred_proba)
        except AttributeError:
            y_pred = estimator.predict(X)
            return roc_auc_score(y, y_pred)

    y_train_str = y_train_res.astype(str)
    score = cross_val_score(clf, X_train_res, y_train_str, cv=cv, scoring=custom_roc_auc).mean()
    return score

# --- Optuna Optimization ---
study_hc2 = optuna.create_study(direction="maximize")
study_hc2.optimize(objective_hc2, n_trials=31)
best_params_hc2 = study_hc2.best_trial.params
print("Best params for HIVE-COTE v2:", best_params_hc2)

* Best Params: {'n_estimators': 455, 'learning_rate': 0.1516395931910056, 'max_features': 'sqrt', 'max_depth': 9, 'subsample': 0.75}
* Score: 0.9678502444785376

In [ ]:
import optuna.visualization as vis
from IPython.display import IFrame, display

In [ ]:
# Plot the parameter importances
plot_param_importances(study_hc2)
#fig_importances.write_html("parameter_importances.html")

In [ ]:
# Plot the optimization history (objective value over
vis.plot_optimization_history(study_hc2)
#fig_history.write_html("optimization_history.html")

In [ ]:
from sklearn.model_selection import cross_val_predict
X_train_res, y_train_res
y_pred = cross_val_predict(KNeighborsTimeSeriesClassifier(**{'n_neighbors': 3, 'weights': 'distance', 'leaf_size': 30}),
                           X_train_res, y_train_res,
                           cv=cv, method="predict_proba")

In [ ]:
y_pred

##### Fit the Model:

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Create a VotingClassifier with the best estimators
clf = KNeighborsTimeSeriesClassifier(n_neighbors=6,distance="euclidean",algorithm="ball_tree",leaf_size=30,weights="distance")

# Fit the VotingClassifier on your resampled training data
clf.fit(X_train_res, y_train_res)

# Evaluate using cross-validation with ROC AUC as the metric.
# If it's a binary classification, use 'roc_auc'.
# For multi-class problems, consider using 'roc_auc_ovr' or 'roc_auc_ovo'
from sklearn.model_selection import cross_val_score
voting_roc_auc = cross_val_score(clf, X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()
print("KNeighborsTimeSeries Classifier CV ROC AUC Score:", voting_roc_auc)

In [ ]:
#obj = sio.dump(clf, "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/clf_KNeighbors_v0.skops")

unknown_types = sio.get_untrusted_types(file="/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/clf_KNeighbors_v0.skops")
# investigate the contents of unknown_types, and only load if you trust
# everything you see.
clf = sio.load("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/clf_KNeighbors_v0.skops", trusted=unknown_types)

In [ ]:
X_test.shape

In [ ]:
#X_test = preprocessing.transform(X_test)
test_predictions_proba = clf.predict_proba(X_test)[:, 1]
test_predictions_proba

In [ ]:
df_subm["rainfall"] = test_predictions_proba
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/clf_KNeighbors_v0_all_data_ext.csv")
df_subm

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)
ax[1].scatter(test_predictions_resampled_cv2,test_predictions_proba)
plt.show()

#### XGBClassifier

In [ ]:
def objective_xgb(trial):
    # Suggest key hyperparameters for XGBClassifier
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    subsample = trial.suggest_float("subsample", 0.5, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 1)
    reg_alpha = trial.suggest_float('reg_alpha', 0.00001, 1.0, log=True)
    reg_lambda = trial.suggest_float('reg_lambda', 0.00001, 1.0, log=True)

    clf = XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda
    )

    score = cross_val_score(clf, X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()
    return score
study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_xgb, n_trials=71) #adjust to 50
best_params_xgb = study_xgb.best_trial.params
print("Best params for XGBoost:", best_params_xgb)

* Parameters: {'n_estimators': 325, 'learning_rate': 0.06278279256080053, 'max_depth': 8, 'subsample': 0.70, 'colsample_bytree': 0.87, 'gamma': 0.17, 'reg_alpha': 0.024, 'reg_lambda': 8.111e-05}
* Score 0.9618260284102824

In [ ]:
# Plot the optimization history (objective value over
vis.plot_optimization_history(study_xgb)
#fig_history.write_html("optimization_history.html")

In [ ]:
# Plot the parameter importances
plot_param_importances(study_xgb)
#fig_importances.write_html("parameter_importances.html")

#### Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

# Build final models using the best parameters from Optuna studies
rf_final = RandomForestClassifier(class_weight='balanced', random_state=42, **best_params_rf)
gb_final = GradientBoostingClassifier(random_state=42, **best_params_gb)
xgb_final = XGBClassifier(use_label_encoder=False, eval_metric="auc", random_state=42, **best_params_xgb)

# Create a VotingClassifier with the best estimators
voting_clf = VotingClassifier(
    estimators=[
        ('rf', rf_final),
        ('gb', gb_final),
        ('xgb', xgb_final)
    ],
    voting='soft'  # or 'soft' if probability estimates are preferred
)

# Fit the VotingClassifier on your resampled training data
voting_clf.fit(X_train_res, y_train_res)

# Evaluate using cross-validation with ROC AUC as the metric.
# If it's a binary classification, use 'roc_auc'.
# For multi-class problems, consider using 'roc_auc_ovr' or 'roc_auc_ovo'
from sklearn.model_selection import cross_val_score
voting_roc_auc = cross_val_score(voting_clf, X_train_res, y_train_res, cv=cv, scoring='roc_auc').mean()
print("Voting Classifier CV ROC AUC Score:", voting_roc_auc)

In [ ]:
obj = sio.dump(voting_clf, "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/voting_v0.skops")

unknown_types = sio.get_untrusted_types(file="/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/voting_v0.skops")
# investigate the contents of unknown_types, and only load if you trust
# everything you see.
voting_clf = sio.load("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S5E3/voting_v0.skops", trusted=unknown_types)

In [ ]:
y_val_pred_proba = voting_clf.predict_proba(X_val)[:, 1]
print('Validation ROC AUC:', roc_auc_score(y_val, y_val_pred_proba),'\n')
print(classification_report(y_val, voting_clf.predict(X_val)))

In [ ]:
X_test = preprocessing.transform(test)
test_predictions_proba = voting_clf.predict_proba(X_test)[:, 1]

In [ ]:
df_subm["rainfall"] = test_predictions_proba
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_voting_v0_all_data_ext.csv")
df_subm

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)
ax[1].scatter(test_predictions_resampled_cv2,test_predictions_proba)
plt.show()

### 4.3 CNN Classifier

In [ ]:
%%capture
!pip install sktime

In [ ]:
from sktime.classification.deep_learning.cnn import CNNClassifier

METRICS = [
          keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
          ]

In [ ]:
resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(10_000).prefetch(2)

In [ ]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())
  print(label.numpy().shape)
  print(features.numpy().shape)

In [ ]:
dataset_validation = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_valid,
                                                                    y_valid,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=2000,
                                                                    shuffle=False
                                                                 )


for batch in dataset_validation.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

In [ ]:
train = np.concatenate((inputs, features), axis=0)
y = np.concatenate((targets, label), axis=0)
train.shape, y.shape

In [ ]:
call_bk = [keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, monitor="val_auc", min_lr=0.000001),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")]

model = GRUFCNNClassifier( hidden_dim=128, gru_layers=2, verbose=True)





#CNNClassifier(n_epochs=8, batch_size=64, kernel_size=3, avg_pool_size=2, n_conv_layers=2, callbacks=call_bk,
#                      verbose=True, loss=keras.losses.BinaryCrossentropy(), metrics=METRICS, random_state=42,
#                      activation='sigmoid', use_bias=True, optimizer=keras.optimizers.Adam(learning_rate=1e-3), filter_sizes=None, padding='auto')

In [ ]:
y_prob_pred = model.fit_predict_proba(train, y, cv=5)

In [ ]:
import tensorflow as tf
from tensorflow import keras

class TimeSeriesPositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1, activation='gelu')(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

# Example Usage
sequence_length = 7 # Example sequence length
embed_dim = 30 # Example embedding dimension.
head_size = 32
num_heads = 4
ff_dim = 128
dropout = 0.1

inputs = keras.layers.Input(shape=(sequence_length, embed_dim))
x = TimeSeriesPositionalEmbedding(sequence_length, embed_dim)(inputs)
x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

## Test SKTime:

In [ ]:
%%capture
!pip install sktime

In [ ]:
from sktime.datasets import load_italy_power_demand

In [ ]:
# Increase display width
pd.set_option("display.width", 1000)

## 2.1 Panel data - sktime data formats

Panel is an abstract data type where the values are observed for:

* instance, e.g., patient
* variable, e.g., blood pressure, body temperature of the patient
* time/index, e.g., January 12, 2023 (usually but not necessarily a time index!)
One value X is: "patient 'A' had blood pressure 'X' on January 12, 2023"

Time series classification, regression, clustering: slices Panel data by instance

Preferred format 1: pd.DataFrame with 2-level MultiIndex, (instance, time) and columns: variables

Preferred format 2: 3D np.ndarray with index (instance, variable, time)

* sktime supports and recognizes multiple data formats for convenience and internal use, e.g., dask, xarray
* abstract data type = "scitype"; in-memory specification = "mtype"
* More information in tutorial on in-memory data representations and data loading

## 2.1.1 Preferred format 1 - pd-multiindex specification

pd-multiindex = pd.DataFrame with 2-level MultiIndex, (instance, time) and columns: variables

In [ ]:
from sktime.datasets import load_italy_power_demand

# load an example time series panel in pd-multiindex mtype
X, _ = load_italy_power_demand(return_type="pd-multiindex")

# renaming columns for illustrative purposes
X.columns = ["total_power_demand"]
X.index.names = ["day_ID", "hour_of_day"]

The Italy power demand dataset has:

* 1096 individual time series instances = single days of total power demand (mean subtracted)
* one single variable per time series instances, total_power_demand
    * total power demand on that day, in that hourly period
    * Since there's only one column, it is a univariate dataset
* individual time series are observed at 24 time (period) points (the same number for all instances)

In the dataset, days are jumbled and of different scope (independent sampling).

* considered independent - because hour_of_day in one sample doesn't affect hour_of_day in another
* for task, e.g., "identify season or weekday/weekend from pattern"

In [ ]:
X

In [ ]:
from sktime.datasets import load_basic_motions

# load an example time series panel in pd-multiindex mtype
X, _ = load_basic_motions(return_type="pd-multiindex")

# renaming columns for illustrative purposes
X.columns = ["accel_1", "accel_2", "accel_3", "gyro_1", "gyro_2", "gyro_3"]
X.index.names = ["trial_no", "timepoint"]

The basic motions dataset has:

* 80 individual time series instances = trials = person engaging in an activity like running, badminton, etc.
* six variables per time series instance, dim_0 to dim_5 (renamed according to the values they represent)
  * 3 accelerometer and 3 gyrometer measurements
  * hence a multivariate dataset
* individual time series are observed at 100 time points (the same number for all instances)

In [ ]:
# The outermost index represents the instance number
# whereas the inner index represents the index of the particular index
# within that instance.
X

In [ ]:
# Select:
# * the fourth variable (gyroscope 1)
# * of the first instance (trial 1 = 0 in python)
# * values at all 100 timestamps
#
X.loc[0, "gyro_1"]

In [ ]:
X.loc[0, "gyro_1"].plot()

## 2.1.2 preferred format 2 - numpy3D

    numpy3D = 3D np.ndarray with index (instance, variable, time)

instance/time index is interpreted as integer

IMPORTANT: unlike pd-multiindex, this assumes:

* all individual series have the same length

* all individual series have the same index

In [ ]:
from sktime.datasets import load_italy_power_demand

# load an example time series panel in numpy mtype
X, _ = load_italy_power_demand(return_type="numpy3D")

In [ ]:
# (num_instances, num_variables, length)
X.shape

In [ ]:
from sktime.datasets import load_basic_motions

# load an example time series panel in numpy mtype
X, _ = load_basic_motions(return_type="numpy3D")

In [ ]:
# (num_instances, num_variables, length)
X.shape

## 2.2.3 Time Series Classification - deployment vignette
Basic deployment vignette for TSC:

load/setup training data, X in a Panel (more specifically numpy3D) format, y as 1D np.ndarray

load/setup new data for prediction (can be done after 3 too)

specify the classifier using sklearn-like syntax

fit classifier to training data, fit(X, y)

predict labels on new data, predict(X_new)

In [ ]:
# steps 1, 2 - prepare osuleaf dataset (train and new)
from sktime.datasets import load_italy_power_demand

X_train, y_train = load_italy_power_demand(split="train", return_type="numpy3D")
X_new, _ = load_italy_power_demand(split="test", return_type="numpy3D")

In [ ]:
# this is in numpy3D format, but could also be pd-multiindex or other
X_train.shape, X_new.shape

In [ ]:
# y is a 1D np.ndarray of labels - same length as number of instances in X_train
y_train.shape

In [ ]:
# step 3 - specify the classifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

# example 1 - 3-NN with simple dynamic time warping distance (requires numba)
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3)

# example 2 - custom distance:
# 3-nearest neighbour classifier with Euclidean distance (on flattened time series)
# (requires scipy)
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels import FlatDist, ScipyDist

eucl_dist = FlatDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=eucl_dist)

In [ ]:
# all classifiers is scikit-learn / scikit-base compatible!
# nested parameter interface via get_params, set_params
clf.get_params()

In [ ]:
# step 4 - fit/train the classifier
clf.fit(X_train, y_train)

In [ ]:
# the classifier is now fitted
clf.is_fitted

In [ ]:
# and we can inspect fitted parameters if we like
clf.get_fitted_params()

In [ ]:
# step 5 - predict labels on new data
y_pred = clf.predict(X_new)
y_prob = clf.predict_proba(X_new)

In [ ]:
# predictions and unique counts, for illustration
unique, counts = np.unique(y_pred, return_counts=True)
unique, counts

In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head

# step 1-- prepare a dataset (multivariate for demonstration)
X_train, y_train = load_arrow_head(split="train", return_type="numpy3D")
X_new, _ = load_arrow_head(split="test", return_type="numpy3D")

In [ ]:
X_train.shape, y_train.shape

In [ ]:
#X_train

In [ ]:
import sktime.classification.deep_learning as dl_clf
from sktime.classification.deep_learning import GRUFCNNClassifier
from sktime.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train", return_type="numpy3D")
X_test, y_test = load_unit_test(split="test", return_type="numpy3D")

In [ ]:
X_train.shape

In [ ]:
cnn = GRUFCNNClassifier( hidden_dim=128, gru_layers=2, verbose=True)
cnn.fit(X_train, y_train)